# Notebook to match "by hand" recipes from menus to recipes in the database
- input files : 
    - "../data/Tools/MenuRecipe.xlsx" which contains a list of recipes taken manually from our list of menus
    - '../data/Recipes/recipes_jow.json' (Jow recipes in french)
- output file : 
    - "../data/Tools/MenuToJowRecipes_v1.xlsx"

In [1]:
import pandas as pd
import re

### Import input data

In [2]:
menu_data = pd.read_excel("../data/Tools/MenuRecipes.xlsx", header = [0])
menu_data.head()

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients
0,burgart_205074_c70.jpg,Burgers,Superfin,"Bœuf (VBF Metzger), Comté AOC, Iceberg, Bacon,..."
1,burgart_205074_c70.jpg,Burgers,Bon & Beau,"Bœuf (VBF Metzger), Cheddar, Sucrine, Bacon, T..."
2,burgart_205074_c70.jpg,Burgers,Dandy,"Bœuf (VBF Metzger), Cheddar fumée, Mâche nant..."
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette..."
4,burgart_205074_c70.jpg,Poke Bowls,Black bowl,"Thon, Riz noir, Avocat, Oignon rouge, Cébette,..."


In [3]:
jow_path = '../data/Recipes/recipes_jow.json'
jow_data = pd.read_json(jow_path)
jow_data = jow_data.transpose()
pd.set_option('display.max_colwidth', None)
jow_data['recettes'] = jow_data.index
jow_data.head()

,cuisson,ingredients,recettes
Yakitori bœuf fromage,10 minutes,"[[80 g, Bœuf (carpaccio)], [100 g, Emmental], [1 càs, Sauce Teriyaki], [70 g, Riz], [6, Pics à brochette]]",Yakitori bœuf fromage
Gnocchis Epinards Gorgonzola,5 minutes,"[[150 g, Gnocchi], [100 g, Epinard (frais)], [30 g, Gorgonzola], [5 cl, Crème liquide]]",Gnocchis Epinards Gorgonzola
Dahl de lentilles corail,plat froid,"[[50 g, Lentilles corail], [80 g, Tomate (chair)], [5 cl, Lait de coco], [1/2 gou., Ail]]",Dahl de lentilles corail
Veggie lasagna,50 minutes,"[[50 g, Lasagnes (cru)], [150 g, Courgette], [150 g, Tomate (purée)], [80 g, Champignons de Paris (frais)], [1/2, Mozzarella (boule)]]",Veggie lasagna
Pasta alla carbonara,9 minutes,"[[100 g, Pâtes (spaghetti)], [3 tran., Pancetta], [1, Œuf], [40 g, Parmesan (morceaux)]]",Pasta alla carbonara


### Match menu recipes to Jow recipes

Let's remove ligatures, e.g. Bœuf -> Boeuf

In [4]:
REGEX_REPLACEMENTS = [(r"\u0153", "oe"), \
                      (r"\u0152", "Oe")]

def remove_ligatures(transcript):
    for old, new in REGEX_REPLACEMENTS:
        transcript = re.sub(old, new, transcript, flags=re.IGNORECASE)
    return transcript

In [5]:
jow_data['recettes'] = jow_data['recettes'].apply(lambda s: remove_ligatures(str(s)))
jow_data['ingredients'] = jow_data['ingredients'].apply(lambda s: remove_ligatures(str(s)))
jow_data.head()

,cuisson,ingredients,recettes
Yakitori bœuf fromage,10 minutes,"[['80 g', 'Boeuf (carpaccio)'], ['100 g', 'Emmental'], ['1 càs', 'Sauce Teriyaki'], ['70 g', 'Riz'], ['6', 'Pics à brochette']]",Yakitori boeuf fromage
Gnocchis Epinards Gorgonzola,5 minutes,"[['150 g', 'Gnocchi'], ['100 g', 'Epinard (frais)'], ['30 g', 'Gorgonzola'], ['5 cl', 'Crème liquide']]",Gnocchis Epinards Gorgonzola
Dahl de lentilles corail,plat froid,"[['50 g', 'Lentilles corail'], ['80 g', 'Tomate (chair)'], ['5 cl', 'Lait de coco'], ['1/2 gou.', 'Ail']]",Dahl de lentilles corail
Veggie lasagna,50 minutes,"[['50 g', 'Lasagnes (cru)'], ['150 g', 'Courgette'], ['150 g', 'Tomate (purée)'], ['80 g', 'Champignons de Paris (frais)'], ['1/2', 'Mozzarella (boule)']]",Veggie lasagna
Pasta alla carbonara,9 minutes,"[['100 g', 'Pâtes (spaghetti)'], ['3 tran.', 'Pancetta'], ['1', 'oeuf'], ['40 g', 'Parmesan (morceaux)']]",Pasta alla carbonara


Let's change to lower case

In [6]:
for col in ["ingredients", "recettes"]:
    jow_data[col] = jow_data[col].apply(lambda s: str(s).lower())
    
jow_data.head(2)

,cuisson,ingredients,recettes
Yakitori bœuf fromage,10 minutes,"[['80 g', 'boeuf (carpaccio)'], ['100 g', 'emmental'], ['1 càs', 'sauce teriyaki'], ['70 g', 'riz'], ['6', 'pics à brochette']]",yakitori boeuf fromage
Gnocchis Epinards Gorgonzola,5 minutes,"[['150 g', 'gnocchi'], ['100 g', 'epinard (frais)'], ['30 g', 'gorgonzola'], ['5 cl', 'crème liquide']]",gnocchis epinards gorgonzola


Let's now match menu recipes to Jow recipes

In [7]:
menu_data['Closest JOW recipe by hand'] = 'Not yet handled'

In [8]:
menu_data

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
0,burgart_205074_c70.jpg,Burgers,Superfin,"Bœuf (VBF Metzger), Comté AOC, Iceberg, Bacon, Compotée d'oignons, Sauce Burgart",Not yet handled
1,burgart_205074_c70.jpg,Burgers,Bon & Beau,"Bœuf (VBF Metzger), Cheddar, Sucrine, Bacon, Tomate Confite, Sauce poivre maison",Not yet handled
2,burgart_205074_c70.jpg,Burgers,Dandy,"Bœuf (VBF Metzger), Cheddar fumée, Mâche nantaise, Bresaola, Aubergine, Oignon rouge, Pickles, Sauce cocktail maison",Not yet handled
3,burgart_205074_c70.jpg,Burgers,Green Chicken,"Filet de poulet, Tomme de Savoie IGP, Roquette, Mortadelle, Parmesan, Sauce verte maison",Not yet handled
4,burgart_205074_c70.jpg,Poke Bowls,Black bowl,"Thon, Riz noir, Avocat, Oignon rouge, Cébette, Concombre, Sésame, Coriandre, Grenade, Sauce douce",Not yet handled
...,...,...,...,...,...
76,fabbrezza_202084_665.jpg,Plat au choix,Pizza Margherita,NaN,Not yet handled
77,fabbrezza_202084_665.jpg,Plat au choix,Penne Arrabiatta,NaN,Not yet handled
78,fabbrezza_202084_665.jpg,Plat au choix,"""Petit"" Lasagne de Tonia",NaN,Not yet handled
79,fabbrezza_202084_665.jpg,Dessert,Baba au Rhum,NaN,Not yet handled


Let's consider one menu recipe

In [9]:
menu_data[menu_data.index==48]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
48,in-bocca-al-lupo_42356_293.jpg,Pizza Bambini (pour les petits),Margherita,"sauce tomate, mozzarella",Not yet handled


In [10]:
jow_data["recettes"][jow_data["recettes"].str.contains("pizza")].values.tolist()

['pizza parma',
 'pizza maison 3 fromages',
 'pizza maison chèvre miel',
 'pizza verde à la courgette',
 'pizza verde à la burrata',
 'pizza printanière',
 'pizza saumon avocat',
 'pizza poire gorgonzola',
 'pizza puttanesca',
 'pizza pesto, tomates & burrata',
 'pizza prosciutto, chèvre & miel',
 'pizza cerise, tapenade & tomme',
 'pizza chorizo & artichaut',
 'pizza margherita',
 'pizza champignons & ricotta',
 'pizza aux anchois',
 'pizza au chorizo',
 'pâte à pizza maison',
 'pizza bianca']

In [11]:
jow_data[jow_data["recettes"]=='pizza margherita'].index[0]

'Pizza Margherita'

In [12]:
menu_data.loc[48, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza margherita'].index[0]

Next menu recipe

In [13]:
menu_data[menu_data.index==76]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
76,fabbrezza_202084_665.jpg,Plat au choix,Pizza Margherita,NaN,Not yet handled


In [14]:
menu_data.loc[76, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza margherita'].index[0]

Next menu recipe

In [15]:
menu_data[menu_data.index==8]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
8,burgart_205074_c70.jpg,Desserts,Fondant au chocolat artisan boulanger,NaN,Not yet handled


In [16]:
jow_data["recettes"][jow_data["recettes"].str.contains("fondant")].values.tolist()

['fondant choco marrons',
 'fondant au chocolat',
 "fondant au chocolat à l'huile d'olive",
 'fondant-flan au chocolat & amande']

In [17]:
jow_data[jow_data["recettes"]=='fondant au chocolat']

,cuisson,ingredients,recettes
Fondant au chocolat,plat froid,"[['200 g', 'chocolat à cuire'], ['125 g', 'beurre demi-sel'], ['130 g', 'sucre (en poudre)'], ['4', 'oeuf'], ['25 g']]",fondant au chocolat


In [18]:
menu_data.loc[8, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='fondant au chocolat'].index[0]

Next menu recipe

In [19]:
menu_data[menu_data.index==7]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
7,burgart_205074_c70.jpg,Desserts,Mousse au chocolat maison,NaN,Not yet handled


In [20]:
jow_data["recettes"][jow_data["recettes"].str.contains("mousse")].values.tolist()

['avocat, pamplemousse & betterave',
 'mousse au chocolat maison',
 "mousse au chocolat à l'orange",
 'salade pamplemousse, avocat & crevettes',
 'asperges & sauce mousseline']

In [21]:
menu_data.loc[7, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='mousse au chocolat maison'].index[0]

Next menu recipe

In [22]:
menu_data[menu_data.index==9]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
9,burgart_205074_c70.jpg,Desserts,Financier artisan boulanger,NaN,Not yet handled


In [23]:
jow_data["recettes"][jow_data["recettes"].str.contains("financier")].values.tolist()

['financier cerises noires & pistache', 'financier à la framboise']

In [24]:
tmp = []
for l in jow_data["recettes"][jow_data["recettes"].str.contains("financier")].values.tolist():
    tmp.append(jow_data[jow_data["recettes"]==l].index[0])

tmp

['Financier cerises noires & pistache', 'Financier à la framboise']

In [25]:
menu_data.loc[9, 'Closest JOW recipe by hand'] = tmp

Next menu recipe

In [26]:
menu_data[menu_data.index==10]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
10,burgart_205074_c70.jpg,Boissons,Expresso,NaN,Not yet handled


In [27]:
jow_data["recettes"][jow_data["recettes"].str.contains("café")].values.tolist()

[]

In [28]:
jow_data["recettes"][jow_data["recettes"].str.contains("expresso")].values.tolist()

[]

In [29]:
jow_data["recettes"][jow_data["recettes"].str.contains("espresso")].values.tolist()

[]

In [30]:
jow_data[jow_data["ingredients"].str.contains("café")].values.tolist()

[['3 heures',
  "[['100 g', 'sucre (en poudre)'], ['facultatif', '1 sac.', 'sucre vanillé'], ['3', 'oeuf'], ['2 càs', 'cacao (en poudre)'], ['250 g', 'mascarpone'], ['20', 'biscuit cuiller'], ['2 càc', 'café'], ['facultatif', '1 càs', 'amaretto']]",
  'tiramisu'],
 ['2 minutes',
  "[['15 cl', 'lait'], ['1', 'café caramel (dose)'], ['1 càc', 'caramel (liquide)']]",
  'caramel macchiato'],
 ['plat froid',
  "[['7 g', 'café'], ['60 g', 'crème glacée vanille'], ['20 g', 'amandes (effilées)']]",
  'affogato']]

In [31]:
menu_data.loc[10, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [32]:
menu_data[menu_data.index==11]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
11,burgart_205074_c70.jpg,Boissons,Café long,NaN,Not yet handled


In [33]:
menu_data.loc[11, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [34]:
menu_data[menu_data.index==12]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
12,burgart_205074_c70.jpg,Boissons,Chocolat chaud,NaN,Not yet handled


In [35]:
jow_data["recettes"][jow_data["recettes"].str.contains("chocolat chaud")].values.tolist()

['chocolat chaud', 'chocolat chaud & marshmallows']

In [36]:
jow_data[jow_data["recettes"]=='chocolat chaud']

,cuisson,ingredients,recettes
Chocolat chaud,plat froid,"[['30 g', 'chocolat à cuire'], ['25 cl', 'lait'], ['1 càc', 'cacao (en poudre)']]",chocolat chaud


In [37]:
menu_data.loc[12, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='chocolat chaud'].index[0]

Next recipe

In [38]:
menu_data[menu_data.index==13]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
13,burgart_205074_c70.jpg,Boissons,Ice Tea,NaN,Not yet handled


In [39]:
jow_data["recettes"][jow_data["recettes"].str.contains("thé")].values.tolist()

[]

In [40]:
jow_data[jow_data["ingredients"].str.contains("thé")].values.tolist()

[]

In [41]:
jow_data["recettes"][jow_data["recettes"].str.contains("ice tea")].values.tolist()

[]

In [42]:
menu_data.loc[13, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [43]:
menu_data[menu_data.index==14]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
14,burgart_205074_c70.jpg,Boissons,Elixia,Limonade Bio,Not yet handled


In [44]:
jow_data["recettes"][jow_data["recettes"].str.contains("limonade")].values.tolist()

[]

In [45]:
jow_data[jow_data["ingredients"].str.contains("limonade")].values.tolist()

[['1 heure',
  "[['10 cl', 'vin rouge'], ['10 cl', 'limonade'], ['1/4', 'orange'], ['1/4', 'citron jaune']]",
  'tinto de verano'],
 ['plat froid',
  "[['150 g', 'pastèque (classique)'], ['30 g', 'concombre'], ['5 cl', 'limonade'], ['1/10 bou.', 'menthe (feuilles)'], ['1/2', 'citron jaune']]",
  'watermelon refresher'],
 ['plat froid',
  "[['70 g', 'pastèque (classique)'], ['20 g', 'concombre'], ['5 cl', 'limonade'], ['1/10 bou.', 'menthe (feuilles)'], ['1/2', 'citron jaune'], ['4 cl', 'vodka']]",
  'vodka watermelon refresher'],
 ['3 heures',
  "[['75 cl', 'vin blanc'], ['60 cl', 'jus de pêche'], ['40 cl', 'limonade'], ['3', 'pêche'], ['3/10 bou.', 'menthe (feuilles)']]",
  'sangria à la pêche']]

In [46]:
menu_data.loc[14, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [47]:
menu_data[menu_data.index==15]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
15,burgart_205074_c70.jpg,Boissons,Alain Milliat,"Orange, Mangue, Pomme",Not yet handled


In [48]:
jow_data["recettes"][jow_data["recettes"].str.contains("jus")].values.tolist()

[]

In [49]:
jow_data[jow_data["ingredients"].str.contains("jus")].values.tolist()

[['35 minutes',
  '[[\'1\', \'poulet (cuisse)\'], [\'1\', \'clémentine\'], [\'1/4\', \'oignon jaune\'], [\'200 g\', \'haricots blancs nature (cuits)\'], [\'1 càs\', "moutarde à l\'ancienne"], [\'5 cl\', "jus d\'orange"]]',
  'poulet clémentine & moutarde'],
 ['plat froid',
  '[[\'5 cl\', \'rhum (blanc)\'], [\'4 cl\', \'lait de coco\'], [\'12 cl\', "jus d\'ananas"]]',
  'piña colada'],
 ['plat froid',
  "[['4 cl', 'vodka'], ['2 cl', 'jus de cranberry'], ['2 cl', 'cointreau'], ['1/4', 'citron vert']]",
  'cosmopolitan'],
 ['plat froid',
  "[['4 cl', 'vodka'], ['12 cl', 'jus de tomate'], ['1/4 càc', 'tabasco'], ['1 pinc.', 'sel au céleri']]",
  'bloody mary'],
 ['plat froid',
  '[[\'4 cl\', \'lait de coco\'], [\'12 cl\', "jus d\'ananas"], [\'1/2\', \'citron vert\'], [\'1/2 càc\', \'sucre vanillé\']]',
  'virgin piña colada'],
 ['3 heures',
  "[['75 cl', 'vin blanc'], ['60 cl', 'jus de pêche'], ['40 cl', 'limonade'], ['3', 'pêche'], ['3/10 bou.', 'menthe (feuilles)']]",
  'sangria à la pêc

In [50]:
menu_data.loc[15, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [51]:
menu_data[menu_data.index==16]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
16,burgart_205074_c70.jpg,Boissons,Bière Corona,NaN,Not yet handled


In [52]:
jow_data["recettes"][jow_data["recettes"].str.contains("bière")].values.tolist()

[]

In [53]:
menu_data.loc[16, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [54]:
menu_data[menu_data.index==17]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
17,Carte-in-seoul.png,Anju,Mandoo (6 pcs),Raviolis maison grillés au porc et légumes,Not yet handled


In [55]:
jow_data["recettes"][jow_data["recettes"].str.contains("mandoo")].values.tolist()

[]

In [56]:
jow_data["recettes"][jow_data["recettes"].str.contains("raviolis")].values.tolist()

['raviolis à la crème de champignons',
 'raviolis, tomate & feta',
 'raviolis aux champignons, chèvre frais & noisette']

In [57]:
jow_data[jow_data["recettes"]=='raviolis aux champignons, chèvre frais & noisette']

,cuisson,ingredients,recettes
"Raviolis aux champignons, chèvre frais & noisette",2 minutes,"[['125 g', 'ravioli (aux champignons)'], ['50 g', 'chèvre frais'], ['20 g', 'noisette'], ['40 g', 'champignons bruns']]","raviolis aux champignons, chèvre frais & noisette"


In [58]:
menu_data.loc[17, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [59]:
menu_data[menu_data.index==18]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
18,Carte-in-seoul.png,Anju,Haemul Jeol,Galette de fruit mer,Not yet handled


In [60]:
jow_data["recettes"][jow_data["recettes"].str.contains("galette")].values.tolist()

['galette forestière',
 'galette saumon',
 'galette complète',
 'galette seguin',
 'galette roquefort',
 'galette des rois maison',
 'galette parma',
 'galette chèvre miel',
 'galette fondue de poireaux & camembert',
 'galette saucisse',
 'galette champignons & oeuf miroir',
 'galette fondue de poireaux & saumon',
 'galette italienne',
 'galette fondue de poireaux & oeuf miroir',
 'galette savoyarde',
 'galette oeuf & chorizo',
 'galette chocolat & amandes express',
 'galette des rois à la pomme',
 'galette de sarrasin',
 'galette épinard chèvre']

In [61]:
jow_data["recettes"][jow_data["recettes"].str.contains("fruit de mer")].values.tolist()

[]

In [62]:
jow_data["recettes"][jow_data["recettes"].str.contains("jeol")].values.tolist()

[]

In [63]:
jow_data[jow_data["ingredients"].str.contains("fruit de mer")].values.tolist()

[]

In [64]:
menu_data.loc[18, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [65]:
menu_data[menu_data.index==19]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
19,Carte-in-seoul.png,Anju,Sewoo kkanpoong (8 pcs),Crevettes frites dans une sauce aigre douce,Not yet handled


In [66]:
jow_data["recettes"][jow_data["recettes"].str.contains("sewoo")].values.tolist()

[]

In [67]:
jow_data[jow_data["recettes"].str.contains("crevette")]

,cuisson,ingredients,recettes
Spaghetti ail & crevette,10 minutes,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'crevette (cuite)'], ['1/2 gou.', 'ail'], ['1 pinc.', ""piment d'espelette""], ['1/10 bou.', 'persil (frais)']]",spaghetti ail & crevette
Crevettes miel sésame,plat froid,"[['100 g', 'crevette (cuite)'], ['70 g', 'riz'], ['1/2 gou.', 'ail'], ['1 càc', 'graines de sésame']]",crevettes miel sésame
Gaspacho repas crevette & avocat,plat froid,"[['30 cl', 'gaspacho'], ['100 g', 'crevette (cuite)'], ['1/2', 'avocat'], ['facultatif', '1/5 bou.', 'basilic (frais)'], ['1 tran.', 'pain de mie']]",gaspacho repas crevette & avocat
Bouillon coco crevettes,plat froid,"[['3 cl', 'lait de coco'], ['1 càc', 'curry (poudre)'], ['1/4', 'bouillon de volaille (cube)'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",bouillon coco crevettes
Blanquette express aux crevettes,plat froid,"[['100 g', 'crevette (cuite)'], ['1/2', 'carotte (frais)'], ['1/2', 'poireau'], ['1 càc', 'fond de veau']]",blanquette express aux crevettes
Ramen aux crevettes,5 minutes,"[['1 càc', 'pâte de curry (rouge)'], ['2 càs', 'lait de coco'], ['1', 'nouilles instantanées (curry)'], ['facultatif', '1/4', 'citron vert'], ['1/10 bou.', 'coriandre (frais)'], ['50 g', 'crevette (cuite)']]",ramen aux crevettes
Risotto crevettes & chorizo,plat froid,"[['70 g', 'riz (arborio)'], ['25 g', 'chorizo (entier)'], ['50 g', 'crevette (cuite)'], ['1/4', 'bouillon de volaille (cube)']]",risotto crevettes & chorizo
Poêlée de crevettes & polenta,8 minutes,"[['50 g', 'polenta'], ['25 cl', 'lait'], ['1 càs', 'parmesan (râpé)'], ['80 g', 'tomate cerise'], ['50 g'], ['1/4 gou.'], ['facultatif', '1/10 bou.']]",poêlée de crevettes & polenta
Pad Thaï crevette,8 minutes,"[['80 g', 'nouilles de riz'], ['1/2 gou.', 'ail'], ['2 càs', 'sauce soja salée'], ['1/2 càs', 'beurre de cacahuète'], ['1/4', 'citron vert'], ['50 g', 'pousses de soja'], ['50 g', 'crevette (cuite)'], ['facultatif', '1/10 bou.', 'coriandre (frais)']]",pad thaï crevette
"Salade de riz, crevettes & avocat",plat froid,"[['60 g', 'riz (mélange)'], ['50 g', 'crevette (cuite)'], ['1/2', 'avocat'], ['1 poignée', 'epinard (frais)'], ['facultatif', '1/10 bou.', 'ciboulette'], ['1/4', 'oignon rouge'], ['1/4', 'citron vert']]","salade de riz, crevettes & avocat"


In [68]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='crevettes teriyaki'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='crevettes miel sésame'].index[0])
tmp

['Crevettes teriyaki', 'Crevettes miel sésame']

In [69]:
menu_data.loc[19, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [70]:
menu_data[menu_data.index==20]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
20,Carte-in-seoul.png,Bibimbap,Dolsot bibimbap (en pierrade au choix),Bœuf,Not yet handled


In [71]:
jow_data[jow_data["recettes"].str.contains("bibimbap")]

,cuisson,ingredients,recettes


In [72]:
jow_data[jow_data["recettes"].str.contains("boeuf")]

,cuisson,ingredients,recettes
Yakitori bœuf fromage,10 minutes,"[['80 g', 'boeuf (carpaccio)'], ['100 g', 'emmental'], ['1 càs', 'sauce teriyaki'], ['70 g', 'riz'], ['6', 'pics à brochette']]",yakitori boeuf fromage
Wok de bœuf légumes croquants,12 minutes,"[['1', 'boeuf (rumsteck)'], ['1', 'carotte (frais)'], ['50 g', 'petits pois (frais)'], ['1/2', 'courgette'], ['facultatif', '1/2 càc', 'graines de sésame'], ['1 cm', 'gingembre (frais)']]",wok de boeuf légumes croquants
Bœuf sauce roquefort,20 minutes,"[['250 g', 'pomme de terre grenaille (surgelées)'], ['1', 'boeuf (steak)'], ['2 cl', 'crème liquide'], ['10 g', 'roquefort']]",boeuf sauce roquefort
Bœuf haricots verts & purée,20 minutes,"[['100 g', 'haricot vert (frais)'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['1', 'boeuf (steak)'], ['20 g', 'beurre']]",boeuf haricots verts & purée
Curry de bœuf thaï,plat froid,"[['1', 'boeuf (steak haché)'], ['1/2', 'carotte (frais)'], ['100 g', 'pomme de terre'], ['1 càc', 'pâte de curry (vert)'], ['10 cl', 'lait de coco'], ['1 càs', 'sauce soja salée'], ['1/4', 'bouillon de volaille (cube)'], ['facultatif', '1/4', 'citron vert']]",curry de boeuf thaï
Ramen au bœuf,7 minutes,"[['1', 'nouilles instantanées (boeuf)'], ['1', 'oeuf'], ['50 g', 'boeuf (carpaccio)'], ['1 càs', 'sauce soja salée'], ['facultatif', '1/10 bou.', 'ciboulette']]",ramen au boeuf
Enchiladas au bœuf,25 minutes,"[['2', 'tortilla (maïs)'], ['1/2', 'poivron rouge'], ['50 g', 'tomate (chair)'], ['1', 'boeuf (steak haché)'], ['1 càs', 'crème fraîche épaisse'], ['20 g', 'cheddar (râpé)'], ['1/2', 'oignon jaune']]",enchiladas au boeuf
Bœuf à la mexicaine,plat froid,"[['1', 'boeuf (steak)'], ['1 càc', 'mélange épices mexicaines'], ['1/4', 'oignon jaune'], ['100 g', 'tomate (chair)'], ['facultatif']]",boeuf à la mexicaine
Bœuf & crème persillée,30 minutes,"[['1', 'boeuf (steak)'], ['1', 'carotte (frais)'], ['200 g', 'pomme de terre'], ['1 càs', 'crème fraîche épaisse'], ['1 càc', 'vinaigre de vin blanc'], ['1/10 bou.', 'persil (frais)'], ['1/4', 'citron jaune'], ['facultatif', '1/2 càc', 'raifort (crème)']]",boeuf & crème persillée
Tagliata de bœuf & courgettes,4 minutes,"[['240 g', 'courgette'], ['1', 'boeuf (rumsteck)'], ['1 càs', 'vinaigre de vin rouge'], ['1/2 càc', 'origan (séché)']]",tagliata de boeuf & courgettes


In [73]:
menu_data.loc[20, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [74]:
menu_data[menu_data.index==21]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
21,Carte-in-seoul.png,Chicken,"YangNyeom chicken (5pcs, 12pcs, 24pcs)",Poulet frit enrobé de sauce + salade,Not yet handled


In [75]:
jow_data[jow_data["recettes"].str.contains("poulet")]

,cuisson,ingredients,recettes
Poulet au miel & haricots,13 minutes,"[['1', 'poulet (escalope)'], ['1 càs', 'miel (liquide)'], ['1/2', 'citron jaune'], ['250 g', 'haricot vert (frais)']]",poulet au miel & haricots
Poulet cacahuète légumes croquants,10 minutes,"[['1 càs', 'beurre de cacahuète'], ['1', 'poulet (escalope)'], ['150 g', 'brocoli (frais)'], ['1 càs', 'sauce soja salée']]",poulet cacahuète légumes croquants
Poulet & poêlée légumes au curry,15 minutes,"[['1', 'poulet (escalope)'], ['1/2', 'courgette'], ['2', 'carotte (frais)'], ['1 càc', 'curry (poudre)'], ['1/2 gou.', 'ail'], ['facultatif', '1/5 bou.', 'coriandre (frais)']]",poulet & poêlée légumes au curry
Poulet satay,plat froid,"[['70 g', 'riz'], ['1', 'poulet (escalope)'], ['1 càs', 'beurre de cacahuète'], ['10 cl', 'lait de coco'], ['1/4', 'oignon rouge'], ['1 pinc.', 'piment de cayenne'], ['1', 'pics à brochette']]",poulet satay
Brochettes de poulet Tsukune,10 minutes,"[['1', 'poulet (escalope)'], ['2 càs', 'sauce soja salée'], ['70 g', 'riz'], ['1/4', 'oeuf'], ['1/4', 'oignon jaune'], ['2 càs', 'chapelure'], ['facultatif', '2', 'pics à brochette'], ['1 càs', 'sucre (en poudre)']]",brochettes de poulet tsukune
...,...,...,...
Poulet basquaise,1 heure,"[['150 g', 'tomate'], ['120 g', 'poivron rouge'], ['1', 'poulet (cuisse)'], ['4 cl', 'vin blanc'], ['1/4', 'oignon jaune'], ['1/10', 'bouquet garni'], ['1/2', 'ail']]",poulet basquaise
Salade au poulet pané,6 minutes,"[['1', 'poulet (pané)'], ['2 poignées', 'salade (mélange)'], ['100 g', 'tomate cerise'], ['15 g', 'parmesan (morceaux)'], ['1 càc', 'crème balsamique']]",salade au poulet pané
Enchiladas poulet & courgettes,34 minutes,"[['2', 'tortilla (blé)'], ['1', 'poulet (escalope)'], ['125 g', 'courgette'], ['8 cl', 'crème liquide'], ['40 g', 'mozzarella (rapé)'], ['1/4', 'oignon jaune']]",enchiladas poulet & courgettes
"Poulet, semoule & ratatouille",10 minutes,"[['1', 'poulet (escalope)'], ['100 g', 'ratatouille'], ['70 g', 'semoule']]","poulet, semoule & ratatouille"


In [76]:
menu_data.loc[21, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [77]:
menu_data[menu_data.index==22]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
22,Carte-in-seoul.png,Classiques,Bulgogi,"Bœuf émincé à la sauce soja, légumes, riz",Not yet handled


In [78]:
jow_data[jow_data["recettes"].str.contains("boeuf")]

,cuisson,ingredients,recettes
Yakitori bœuf fromage,10 minutes,"[['80 g', 'boeuf (carpaccio)'], ['100 g', 'emmental'], ['1 càs', 'sauce teriyaki'], ['70 g', 'riz'], ['6', 'pics à brochette']]",yakitori boeuf fromage
Wok de bœuf légumes croquants,12 minutes,"[['1', 'boeuf (rumsteck)'], ['1', 'carotte (frais)'], ['50 g', 'petits pois (frais)'], ['1/2', 'courgette'], ['facultatif', '1/2 càc', 'graines de sésame'], ['1 cm', 'gingembre (frais)']]",wok de boeuf légumes croquants
Bœuf sauce roquefort,20 minutes,"[['250 g', 'pomme de terre grenaille (surgelées)'], ['1', 'boeuf (steak)'], ['2 cl', 'crème liquide'], ['10 g', 'roquefort']]",boeuf sauce roquefort
Bœuf haricots verts & purée,20 minutes,"[['100 g', 'haricot vert (frais)'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['1', 'boeuf (steak)'], ['20 g', 'beurre']]",boeuf haricots verts & purée
Curry de bœuf thaï,plat froid,"[['1', 'boeuf (steak haché)'], ['1/2', 'carotte (frais)'], ['100 g', 'pomme de terre'], ['1 càc', 'pâte de curry (vert)'], ['10 cl', 'lait de coco'], ['1 càs', 'sauce soja salée'], ['1/4', 'bouillon de volaille (cube)'], ['facultatif', '1/4', 'citron vert']]",curry de boeuf thaï
Ramen au bœuf,7 minutes,"[['1', 'nouilles instantanées (boeuf)'], ['1', 'oeuf'], ['50 g', 'boeuf (carpaccio)'], ['1 càs', 'sauce soja salée'], ['facultatif', '1/10 bou.', 'ciboulette']]",ramen au boeuf
Enchiladas au bœuf,25 minutes,"[['2', 'tortilla (maïs)'], ['1/2', 'poivron rouge'], ['50 g', 'tomate (chair)'], ['1', 'boeuf (steak haché)'], ['1 càs', 'crème fraîche épaisse'], ['20 g', 'cheddar (râpé)'], ['1/2', 'oignon jaune']]",enchiladas au boeuf
Bœuf à la mexicaine,plat froid,"[['1', 'boeuf (steak)'], ['1 càc', 'mélange épices mexicaines'], ['1/4', 'oignon jaune'], ['100 g', 'tomate (chair)'], ['facultatif']]",boeuf à la mexicaine
Bœuf & crème persillée,30 minutes,"[['1', 'boeuf (steak)'], ['1', 'carotte (frais)'], ['200 g', 'pomme de terre'], ['1 càs', 'crème fraîche épaisse'], ['1 càc', 'vinaigre de vin blanc'], ['1/10 bou.', 'persil (frais)'], ['1/4', 'citron jaune'], ['facultatif', '1/2 càc', 'raifort (crème)']]",boeuf & crème persillée
Tagliata de bœuf & courgettes,4 minutes,"[['240 g', 'courgette'], ['1', 'boeuf (rumsteck)'], ['1 càs', 'vinaigre de vin rouge'], ['1/2 càc', 'origan (séché)']]",tagliata de boeuf & courgettes


In [79]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='wok de boeuf légumes croquants'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='boeuf thaï'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='boeuf sauté au chou'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='aiguillettes de boeuf & légumes poêlés'].index[0])
tmp

['Wok de bœuf légumes croquants',
 'Bœuf thaï',
 'Bœuf sauté au chou',
 'Aiguillettes de bœuf & légumes poêlés']

In [80]:
menu_data.loc[22, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [81]:
menu_data[menu_data.index==23]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
23,Carte-in-seoul.png,Classiques,Jeyuk,"Porc sauté, légumes, riz - soja ou piquant",Not yet handled


In [82]:
jow_data[jow_data["recettes"].str.contains("jeyuk")]

,cuisson,ingredients,recettes


In [83]:
jow_data[jow_data["recettes"].str.contains("porc")]

,cuisson,ingredients,recettes
Sauté de porc à l'ananas,30 minutes,"[['1/4', 'porc (filet mignon)'], ['1 tran.', 'ananas au sirop (tranches)'], ['1/4', 'échalote'], ['1/4 gou.', 'ail'], ['70 g', 'riz'], ['1/2 càc', 'curry (poudre)'], ['2 cl', 'vin blanc'], ['1/4', 'bouillon de volaille (cube)']]",sauté de porc à l'ananas
Porc au caramel,plat froid,"[['1/4', 'porc (filet mignon)'], ['70 g', 'riz'], ['2 càs', 'sauce soja salée'], ['1/2 càs', 'vinaigre de xérès'], ['1 càc', 'sucre (en poudre)'], ['1 cm', 'gingembre (frais)']]",porc au caramel
Nouilles de riz & porc sauté,7 minutes,"[['80 g', 'nouilles de riz'], ['80 g', 'chair à saucisse'], ['50 g', 'concombre'], ['20 g', 'cacahuète'], ['1 pinc.', ""piment d'espelette""], ['2 càs', 'sauce soja salée'], ['facultatif', '1/10 bou.', 'menthe (feuilles)']]",nouilles de riz & porc sauté
Côtes de porc au curcuma & tomates,40 minutes,"[['1', 'côte de porc'], ['1/2', 'citron vert'], ['3', 'tomate'], ['1/10 bou.', 'persil (frais)'], ['1/4 càc'], ['1/4 càs']]",côtes de porc au curcuma & tomates
Aubergine braisée au porc,15 minutes,"[['150 g', 'aubergine'], ['60 g', 'chair à saucisse'], ['1 gou.', 'ail'], ['5 g', 'gingembre (frais)'], ['1 càc', 'sucre (en poudre)'], ['1 càs', 'sauce soja salée'], ['70 g', 'riz'], ['facultatif', '1/10 bou.', 'ciboulette']]",aubergine braisée au porc
Côte de porc miel curry & patate douce,25 minutes,"[['1', 'côte de porc'], ['250 g', 'patate douce'], ['1 càc', 'miel (liquide)'], ['1/2 càc', 'curry (poudre)'], ['1/4'], ['facultatif', '1/2 càc']]",côte de porc miel curry & patate douce
Côte de porc & tomates cerises rôties,10 minutes,"[['1', 'côte de porc'], ['100 g', 'tomate cerise'], ['70 g', 'riz'], ['1 pinc.', 'herbes de provence'], ['facultatif', '1/10 bou.']]",côte de porc & tomates cerises rôties


In [84]:
menu_data.loc[23, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='porc au caramel'].index[0]

Next recipe

In [85]:
menu_data[menu_data.index==24]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
24,Carte-in-seoul.png,Classiques,Ojing-oe,"Calamar sauté, légumes, riz - soja ou piquant )",Not yet handled


In [86]:
jow_data[jow_data["recettes"].str.contains("ojing")]

,cuisson,ingredients,recettes


In [87]:
jow_data[jow_data["recettes"].str.contains("calamar")]

,cuisson,ingredients,recettes


In [88]:
jow_data[jow_data["ingredients"].str.contains("calamar")]

,cuisson,ingredients,recettes


In [89]:
menu_data.loc[24, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [90]:
menu_data[menu_data.index==27]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
27,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,La César,"poulet grillé, tomates cerise, œuf, romaine, parmesan",Not yet handled


In [91]:
jow_data[jow_data["recettes"].str.contains("césar")]

,cuisson,ingredients,recettes
Salade César,13 minutes,"[['1', 'poulet (escalope)'], ['2 poignées', 'salade (coeur de laitue)'], ['20 g', 'parmesan (morceaux)'], ['1 tran.', 'pain de campagne (tranché)'], ['1/4', 'oeuf'], ['1/4 càs', 'moutarde'], ['1/4 gou.', 'ail'], ['4 cl', 'huile de tournesol']]",salade césar


In [92]:
menu_data.loc[27, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='salade césar'].index[0]

Next recipe

In [93]:
menu_data[menu_data.index==28]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
28,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,La Biquette de Rocamadour,mielleuse et craquante,Not yet handled


In [94]:
jow_data[jow_data["recettes"].str.contains("chèvre")]

,cuisson,ingredients,recettes
Tartine chèvre figue jambon,10 minutes,"[['2', 'figue'], ['2 tran.', 'pain de campagne (tranché)'], ['2 càs', 'chèvre frais'], ['2 tran.', 'jambon cru'], ['1 càc', 'miel (liquide)'], ['10 g', 'noix'], ['1 poignée', 'salade (mélange)']]",tartine chèvre figue jambon
Toasts chèvre miel,10 minutes,"[['2', 'chèvre (crottin)'], ['2 tran.', 'pain de campagne (tranché)'], ['10 g', 'noix'], ['2 càc', 'miel (liquide)'], ['1 poignée', 'salade (mélange)']]",toasts chèvre miel
Tourte chèvre épinards,40 minutes,"[['1 gou.', 'ail'], ['2', 'pâte feuilletée'], ['250 g', 'chèvre frais'], ['3', 'oeuf'], ['250 g', 'epinards (surgelés)'], ['60 g', 'parmesan (râpé)'], ['facultatif', '2 càs', 'pignons de pin'], ['2 càs', 'sauce pesto']]",tourte chèvre épinards
"Salade de betterave, chèvre & noix",10 minutes,"[['150 g', 'betterave (cuite)'], ['50 g', 'quinoa (cru)'], ['15 g', 'noix'], ['30 g', 'chèvre frais'], ['2 poignées', 'salade (mélange)'], ['2 càc', 'vinaigre de cidre']]","salade de betterave, chèvre & noix"
Pizza maison chèvre miel,10 minutes,"[['1', 'pâte à pizza'], ['2 g', 'romarin (séché)'], ['2 càc', 'miel (liquide)'], ['100 g', 'chèvre frais']]",pizza maison chèvre miel
"Quiche oignons, chèvre & miel",40 minutes,"[['120 g', 'chèvre (bûche)'], ['2 càc', 'miel (liquide)'], ['3', 'oignon jaune'], ['3', 'oeuf'], ['100 g'], ['1'], ['10 cl'], ['facultatif', '6 pinc.']]","quiche oignons, chèvre & miel"
Quiche brocoli saumon & chèvre,35 minutes,"[['1', 'pâte brisée'], ['1', 'brocoli (frais)'], ['4 tran.', 'saumon (fumé)'], ['100 g', 'chèvre frais'], ['100 g'], ['10 cl'], ['3']]",quiche brocoli saumon & chèvre
Wrap courgette chèvre,plat froid,"[['2 càs', 'chèvre frais'], ['1/2', 'courgette'], ['2 poignées', 'salade (mélange mâche-roquette)'], ['1', 'tortilla (blé)']]",wrap courgette chèvre
Quiche betterave chèvre noix,50 minutes,"[['3', 'oeuf'], ['17 cl', 'crème liquide'], ['1', 'pâte brisée'], ['100 g', 'chèvre frais'], ['400 g', 'betterave (cuite)'], ['20 g', 'noix'], ['facultatif', '4 càc', ""confits d'oignons""]]",quiche betterave chèvre noix
Brick chèvre poire,15 minutes,"[['1', 'feuille de brick'], ['2 poignées', 'salade (mélange)'], ['1/2', 'poire'], ['15 g', 'pistaches (grillées)'], ['1/2 càc', 'miel (liquide)'], ['40 g', 'chèvre frais'], ['facultatif', '1 pinc.', 'romarin (séché)']]",brick chèvre poire


In [95]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='salade de chèvre chaud'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='salade chèvre, miel & pignons'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='salade aux croustillants de chèvre'].index[0])
tmp

['Salade de chèvre chaud',
 'Salade chèvre, miel & pignons',
 'Salade aux croustillants de chèvre']

In [96]:
menu_data.loc[28, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [97]:
menu_data[menu_data.index==29]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
29,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Salades,Végé Bowl,"avocat, choux kale frit, concombre, edamame, radis, tomate et tofu mariné aux sésames",Not yet handled


In [98]:
jow_data[jow_data["recettes"].str.contains("bowl")]

,cuisson,ingredients,recettes
Primavera Poke bowl,10 minutes,"[['1', 'saumon (cru)'], ['1/2', 'avocat'], ['70 g', 'riz'], ['2 càs', 'sauce soja salée'], ['2 càs', 'vinaigre de riz'], ['10 g', 'radis'], ['facultatif', '1/5 bou.', 'coriandre (frais)'], ['20 g', 'concombre']]",primavera poke bowl
Rainbowl,plat froid,"[['1/2', 'avocat'], ['100 g', 'maïs (conserve)'], ['1/5 bou.', 'coriandre (frais)'], ['1/5', 'oignon rouge'], ['100 g', 'tomate cerise'], ['1/4', 'citron jaune']]",rainbowl
Comfort Bowl,15 minutes,"[['150 g', 'patate douce'], ['1', 'oeuf'], ['1 poignée', 'salade (roquette)'], ['70 g', 'riz (mélange)'], ['1/4', 'citron jaune']]",comfort bowl
Rice Power Bowl,10 minutes,"[['70 g', 'riz'], ['1/2', 'avocat'], ['1', 'oeuf'], ['facultatif', '1/10 bou.', 'ciboulette'], ['1 càc', 'vinaigre de vin rouge']]",rice power bowl
Cuban rice bowl,10 minutes,"[['70 g', 'riz'], ['1', 'poulet (escalope)'], ['1/2 gou.', 'ail'], ['1 càs', 'sauce soja salée'], ['facultatif', '1/4', 'citron vert'], ['facultatif', '1/10 bou.', 'coriandre (frais)'], ['1/2 càc', 'cumin (moulu)'], ['100 g', 'haricots rouges (cuits)']]",cuban rice bowl
Sésame bowl,15 minutes,"[['40 g', 'quinoa (cru)'], ['1/4', 'aubergine'], ['1/4', 'brocoli (frais)'], ['1/2 càc', 'graines de sésame'], ['1/2 càs', 'beurre de cacahuète'], ['1/4', 'citron jaune'], ['70 g', 'pois chiches (cuits)']]",sésame bowl
Bowl poire & chocolat,5 minutes,"[['1', 'poire'], ['2 càs', 'fromage blanc'], ['50 g', 'chocolat à cuire'], ['5 cl', 'crème liquide'], ['20 g', 'palet breton']]",bowl poire & chocolat
Sashimi bowl,10 minutes,"[['1', 'saumon (cru)'], ['1/4', 'oignon rouge'], ['1/4', 'citron jaune'], ['1/2 càs', 'câpres'], ['1/2 càs', ""huile d'olive""], ['70 g', 'riz (à sushi)']]",sashimi bowl
Vegan poke bowl,10 minutes,"[['70 g', 'riz (complet)'], ['70 g', 'tofu (nature)'], ['1 càs', 'sauce soja salée'], ['1/2', 'avocat'], ['50 g', 'concombre'], ['1 càs', 'yaourt végétal (coco)']]",vegan poke bowl
Mexican bowl,10 minutes,"[['1', 'poulet (escalope)'], ['1 càc', 'mélange épices mexicaines'], ['1/4', 'avocat'], ['60 g', 'maïs (conserve)'], ['60 g', 'haricots rouges (cuits)'], ['70 g', 'riz'], ['facultatif', '1/10 bou.', 'coriandre (frais)']]",mexican bowl


In [99]:
menu_data.loc[29, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='vegan poke bowl'].index[0]

Next recipe

In [100]:
menu_data[menu_data.index==30]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
30,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Moussaka végétarienne,aux gombos,Not yet handled


In [101]:
jow_data[jow_data["recettes"].str.contains("moussaka")]

,cuisson,ingredients,recettes


In [102]:
jow_data[jow_data["ingredients"].str.contains("gomba")]

,cuisson,ingredients,recettes


In [103]:
menu_data.loc[30, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [104]:
menu_data[menu_data.index==32]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
32,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Risotto primavera,"asperges, artichaut et petit pois",Not yet handled


In [105]:
jow_data[jow_data["recettes"].str.contains("risotto")]

,cuisson,ingredients,recettes
Risotto aux Saint-Jacques & poireaux,plat froid,"[['70 g', 'riz (arborio)'], ['1', 'poireau'], ['1/4', 'bouillon de légumes (cube)'], ['10 g', 'beurre']]",risotto aux saint-jacques & poireaux
Risotto aux poireaux,plat froid,"[['70 g', 'riz (arborio)'], ['1', 'poireau'], ['1/4', 'bouillon de légumes (cube)'], ['10 g', 'beurre']]",risotto aux poireaux
Risotto au pesto,plat froid,"[['70 g', 'riz (arborio)'], ['1/5 bou.', 'basilic (frais)'], ['1 càs', 'parmesan (râpé)'], ['15 g', 'noix de cajou'], ['1/4', 'échalote'], ['2 cl', 'vin blanc'], ['1/4', 'bouillon de volaille (cube)'], ['1/4', 'ail']]",risotto au pesto
Risotto de coquillettes,plat froid,"[['90 g', 'pâtes (coquillettes)'], ['1 tran.', 'jambon blanc'], ['1/4', 'échalote'], ['20 g', 'comté'], ['1/2 càs'], ['1/4'], ['2 cl']]",risotto de coquillettes
Risotto crevettes & chorizo,plat froid,"[['70 g', 'riz (arborio)'], ['25 g', 'chorizo (entier)'], ['50 g', 'crevette (cuite)'], ['1/4', 'bouillon de volaille (cube)']]",risotto crevettes & chorizo
Risotto de quinoa au brocoli,25 minutes,"[['1', 'poulet (escalope)'], ['50 g', 'quinoa (cru)'], ['1/4', 'bouillon de volaille (cube)'], ['150 g', 'brocoli (frais)'], ['1/4 gou.', 'ail'], ['1 càs', 'parmesan (râpé)'], ['1/4', 'oignon jaune'], ['20 g', 'noix de cajou']]",risotto de quinoa au brocoli
Risotto aux asperges,plat froid,"[['70 g', 'riz (arborio)'], ['100 g', 'asperges vertes (fraîches)'], ['1/4', 'bouillon de légumes (cube)'], ['1 càs', 'parmesan (râpé)']]",risotto aux asperges
Risotto aux courgettes,plat froid,"[['70 g', 'riz (arborio)'], ['100 g', 'courgette'], ['1/4', 'bouillon de légumes (cube)'], ['1 càs', 'parmesan (râpé)']]",risotto aux courgettes
Risotto comté & saucisse à l'ancienne,plat froid,"[['70 g', 'riz (arborio)'], ['1/4', 'échalote'], ['20 g', 'comté'], ['100 g', 'saucisse (fumée)']]",risotto comté & saucisse à l'ancienne
Risotto aux poivrons,plat froid,"[['70 g', 'riz (arborio)'], ['1/2', 'poivron rouge'], ['1/4', 'bouillon de légumes (cube)'], ['1 càs', 'parmesan (râpé)']]",risotto aux poivrons


In [106]:
menu_data.loc[32, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='risotto aux asperges'].index[0]

Next recipe

In [107]:
menu_data[menu_data.index==33]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
33,Carte-Restaurant-Nelsons-Paris-Jour-2-1024x724.jpg,Végé pour tous,Truffade des montagnes,salade,Not yet handled


In [108]:
jow_data[jow_data["recettes"].str.contains("truffade")]

,cuisson,ingredients,recettes


In [109]:
jow_data[jow_data["recettes"].str.contains("tartiflette")]

,cuisson,ingredients,recettes
Tartiflette express,35 minutes,"[['100 g', 'reblochon'], ['250 g', 'pomme de terre'], ['50 g', 'lardons'], ['1/2', 'oignon jaune'], ['2 cl', 'vin blanc'], ['1 càs', 'crème fraîche épaisse']]",tartiflette express


In [110]:
menu_data.loc[33, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='tartiflette express'].index[0]

Next recipe

In [111]:
menu_data[menu_data.index==37]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
37,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Starters,"Gaspacho tomate, pastèque et crabe","Tomato, watermelon, coriander and chive oil",Not yet handled


In [112]:
jow_data[jow_data["recettes"].str.contains("gaspacho")]

,cuisson,ingredients,recettes
Gaspacho de petits pois à la menthe,12 minutes,"[['200 g', 'petits pois (surgelés)'], ['1/5 bou.', 'menthe (feuilles)'], ['1/2', 'échalote'], ['1/2 pinc.', ""piment d'espelette""], ['2 càs', 'lait de coco']]",gaspacho de petits pois à la menthe
Gaspacho tomate fraise,plat froid,"[['3', 'tomate'], ['4', 'fraise'], ['1/10 bou.', 'basilic (frais)'], ['1/2', 'burrata'], ['1/4 gou.'], ['1/4'], ['1']]",gaspacho tomate fraise
Gaspacho repas crevette & avocat,plat froid,"[['30 cl', 'gaspacho'], ['100 g', 'crevette (cuite)'], ['1/2', 'avocat'], ['facultatif', '1/5 bou.', 'basilic (frais)'], ['1 tran.', 'pain de mie']]",gaspacho repas crevette & avocat
Gaspacho de betterave,plat froid,"[['150 g', 'betterave (cuite)'], ['120 g', 'concombre'], ['1/4', 'citron jaune'], ['40 g', 'chèvre frais'], ['facultatif']]",gaspacho de betterave
Gaspacho andalou,plat froid,"[['150 g', 'tomate'], ['1/2', 'poivron rouge'], ['1/4', 'concombre'], ['1/4', 'oignon rouge'], ['1/4 gou.', 'ail']]",gaspacho andalou
Gaspacho verde,plat froid,"[['200 g', 'courgette'], ['1/4', 'oignon jaune'], ['2 càs', 'yaourt grec'], ['1/4 gou.', 'ail'], ['1/10 bou.', 'menthe (feuilles)'], ['1/2', 'avocat']]",gaspacho verde
Gaspacho citron & avocat,plat froid,"[['30 cl', 'gaspacho'], ['1/2', 'avocat'], ['facultatif', '1/5 bou.', 'basilic (frais)'], ['30 g', 'sorbet citron vert']]",gaspacho citron & avocat


In [113]:
menu_data.loc[37, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='gaspacho repas crevette & avocat'].index[0]

Next recipe

In [114]:
menu_data[menu_data.index==38]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
38,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Starters,Carpaccio Thon Sésame,"Tuna carpaccio, miso, mango and lime salsa, chilli",Not yet handled


In [115]:
jow_data[jow_data["recettes"].str.contains("thon")]

,cuisson,ingredients,recettes
Brandade de thon,plat froid,"[['200 g', 'pomme de terre'], ['50 g', 'thon (conserve)'], ['1/4 gou.', 'ail'], ['facultatif', '1/10 bou.', 'persil (frais)'], ['8 cl', 'lait'], ['2 càs', 'chapelure']]",brandade de thon
Muffins thon & avocat,plat froid,"[['1', 'english muffin'], ['40 g', 'thon (conserve)'], ['40 g', 'fromage frais'], ['1/2', 'avocat'], ['1/2 poignée', 'epinard (frais)']]",muffins thon & avocat
"Pâtes thon, citron & ricotta",12 minutes,"[['100 g', 'pâtes (linguine)'], ['1 càs', 'ricotta'], ['50 g', 'thon au citron'], ['1/4', 'citron jaune'], ['1 càs', 'crème liquide']]","pâtes thon, citron & ricotta"
Salade au thon & œuf mollet,7 minutes,"[['50 g', 'thon (conserve)'], ['1', 'oeuf'], ['1 càc', ""moutarde à l'ancienne""], ['2 poignées', 'salade (mâche)'], ['1 càc', 'vinaigre de cidre']]",salade au thon & oeuf mollet
"Salade semoule, tomate & thon",5 minutes,"[['50 g', 'semoule'], ['100 g', 'tomate cerise'], ['60 g', 'poivron vert'], ['1/4', 'citron jaune'], ['facultatif', '1/10 bou.', 'persil (frais)'], ['50 g', 'thon (conserve)']]","salade semoule, tomate & thon"
Salade de riz thon & coriandre,10 minutes,"[['70 g', 'riz'], ['50 g', 'thon (conserve)'], ['1/10 bou.', 'coriandre (frais)'], ['1/8', 'oignon rouge']]",salade de riz thon & coriandre
Gratin de pâtes au thon,30 minutes,"[['100 g', 'pâtes (fusilli)'], ['5 cl', 'crème liquide'], ['80 g', 'thon (conserve)'], ['15 g', 'fromage râpé']]",gratin de pâtes au thon
"Bruschetta thon, tomate & œuf",9 minutes,"[['1', 'oeuf'], ['2 tran.', 'pain de campagne (tranché)'], ['70 g', 'tomate'], ['70 g', 'thon (conserve)'], ['10 g', 'olive noire'], ['1 càs', 'mayonnaise']]","bruschetta thon, tomate & oeuf"
Salade de thon & haricots blancs,10 minutes,"[['80 g', 'thon (conserve)'], ['1 poignée', 'salade (roquette)'], ['60 g', 'mélange céréales'], ['50 g', 'haricots blancs nature (cuits)'], ['1/8'], ['1/2']]",salade de thon & haricots blancs
"Quiche thon, tomate & moutarde",50 minutes,"[['2 càs', 'moutarde'], ['150 g', 'thon (conserve)'], ['3', 'oeuf'], ['10 cl', 'lait'], ['10 cl', 'crème liquide'], ['300 g', 'tomate'], ['1', 'pâte brisée']]","quiche thon, tomate & moutarde"


In [116]:
menu_data.loc[38, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [117]:
menu_data[menu_data.index==40]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
40,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Mains,Paris London Rumsteak,"(240g) French beef, onion rings, café de Paris butter, pepper sauce",Not yet handled


In [118]:
jow_data[jow_data["recettes"].str.contains("steak")]

,cuisson,ingredients,recettes
Rumsteak à l'italienne,4 minutes,"[['1', 'boeuf (rumsteck)'], ['2 poignées', 'salade (roquette)'], ['30 g', 'parmesan (morceaux)'], ['1 càc', 'sauce pesto'], ['1 càs', 'pignons de pin']]",rumsteak à l'italienne
Steak & cheddar polenta,15 minutes,"[['50 g', 'polenta'], ['10 g', 'cheddar (râpé)'], ['25 cl', 'lait'], ['1', 'boeuf (steak)'], ['1/2', 'oignon jaune'], ['50 g', 'tomate (chair)'], ['1/2', 'poivron rouge'], ['1/4 gou.', 'ail']]",steak & cheddar polenta
Steak de chou-fleur,25 minutes,"[['1/2', 'chou-fleur (frais)'], ['1/4 càc', 'harissa (pâte)'], ['1 càs', ""sirop d'érable""], ['1/4 gou.', 'ail'], ['1/2 càc', 'paprika'], ['1/8', 'citron jaune'], ['facultatif', '1/10 bou.', 'persil (frais)']]",steak de chou-fleur
Steak haché & pommes dauphines,15 minutes,"[['120 g', 'pomme dauphine'], ['1', 'boeuf (steak haché)']]",steak haché & pommes dauphines
Steak haché & pommes noisettes,15 minutes,"[['120 g', 'pomme noisette'], ['1', 'boeuf (steak haché)']]",steak haché & pommes noisettes
Steak haché sauce au roquefort & purée,plat froid,"[['1', 'boeuf (steak haché)'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre']]",steak haché sauce au roquefort & purée
"Steak haché, purée & haricots verts",plat froid,"[['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['1/2', 'échalote'], ['100 g', 'haricot vert (frais)'], ['3 cl'], ['20 g']]","steak haché, purée & haricots verts"
Steak haché aux échalotes & pommes sautées,17 minutes,"[['1', 'boeuf (steak haché)'], ['200 g', 'pommes de terre rissolées'], ['20 g', 'beurre'], ['1/2', 'échalote']]",steak haché aux échalotes & pommes sautées
Steak haché à l'échalote & purée,plat froid,"[['1', 'boeuf (steak haché)'], ['250 g', 'pomme de terre'], ['1/2', 'échalote'], ['3 cl', 'lait']]",steak haché à l'échalote & purée
Steak de chou-fleur & haricots verts,24 minutes,"[['1/2', 'chou-fleur (frais)'], ['1 càs', ""sirop d'érable""], ['1/4 gou.', 'ail'], ['1/8', 'citron jaune'], ['100 g', 'haricot vert (frais)'], ['1 càc', 'paprika']]",steak de chou-fleur & haricots verts


In [119]:
menu_data.loc[40, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [120]:
menu_data[menu_data.index==41]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
41,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Mains,Miso Salmon,"Raw and cooked white asparagus, artichoke mousse",Not yet handled


In [121]:
jow_data[jow_data["recettes"].str.contains("miso")]

,cuisson,ingredients,recettes


In [122]:
jow_data[jow_data["recettes"].str.contains("saumon")]

,cuisson,ingredients,recettes
Saumon mi-cuit & riz,10 minutes,"[['1', 'saumon (frais)'], ['90 g', 'riz']]",saumon mi-cuit & riz
Salade saumon mangue avocat,plat froid,"[['2 tran.', 'saumon (fumé)'], ['1/4', 'mangue'], ['2 poignées', 'epinard (frais)'], ['1 cm', 'gingembre (frais)'], ['1 càs', 'sauce soja salée'], ['1/2', 'avocat'], ['1/2', 'citron vert'], ['facultatif', '1/2 càc', 'graines de sésame']]",salade saumon mangue avocat
Wrap Saumon,plat froid,"[['2 tran.', 'saumon (fumé)'], ['1', 'tortilla (blé)'], ['1/10', 'concombre'], ['40 g', 'fromage frais'], ['1/2 poignée', 'epinard (frais)'], ['1/8', 'citron jaune']]",wrap saumon
Papillote saumon couscous,17 minutes,"[['50 g', 'couscous aux épices'], ['1', 'saumon (frais)'], ['30 g', 'tomate séchée'], ['1', 'oignon nouveau']]",papillote saumon couscous
Saumon mi-cuit & salade citronnée,10 minutes,"[['40 g', 'concombre'], ['1', 'saumon (frais)'], ['80 g', 'mélange céréales'], ['1/4', 'citron jaune'], ['3/10 bou.']]",saumon mi-cuit & salade citronnée
...,...,...,...
Salade de couscous & saumon grillé,9 minutes,"[['1', 'saumon (frais)'], ['70 g', 'semoule'], ['1/4', 'concombre'], ['30 g', 'feta'], ['1/10 bou.', 'menthe (feuilles)'], ['facultatif', '1/4', 'citron jaune']]",salade de couscous & saumon grillé
Papillote saumon pesto & légumes d'été,30 minutes,"[['1', 'saumon (frais)'], ['120 g', 'haricot vert (surgelé)'], ['100 g', 'tomate cerise'], ['1 càs', 'sauce pesto']]",papillote saumon pesto & légumes d'été
"Filet de saumon en croûte, tomates & courgettes",30 minutes,"[['600 g', 'filet de saumon'], ['600 g', 'courgette'], ['400 g', 'tomate cerise'], ['80 g', 'parmesan (râpé)'], ['60 g', 'beurre'], ['40 g', 'chapelure']]","filet de saumon en croûte, tomates & courgettes"
"Saumon, boulgour & ratatouille",10 minutes,"[['1', 'saumon (frais)'], ['100 g', 'ratatouille'], ['70 g', 'boulgour']]","saumon, boulgour & ratatouille"


In [123]:
jow_data[jow_data["recettes"].str.contains("asperge")]

,cuisson,ingredients,recettes
Œuf à la coque & asperges vertes,7 minutes,"[['1', 'oeuf'], ['100 g', 'asperges vertes (fraîches)']]",oeuf à la coque & asperges vertes
Œuf cocotte asperges et parmesan,15 minutes,"[['1', 'oeuf'], ['1 càs', 'crème fraîche épaisse'], ['100 g', 'asperges vertes (fraîches)'], ['15 g', 'parmesan (râpé)']]",oeuf cocotte asperges et parmesan
Risotto aux asperges,plat froid,"[['70 g', 'riz (arborio)'], ['100 g', 'asperges vertes (fraîches)'], ['1/4', 'bouillon de légumes (cube)'], ['1 càs', 'parmesan (râpé)']]",risotto aux asperges
Asperges & vinaigrette à l'estragon,7 minutes,"[['250 g', 'asperges vertes (fraîches)'], ['1', 'oeuf'], ['1 càc', 'moutarde'], ['1/4', 'échalote'], ['2 tran.'], ['1 càc'], ['1 g']]",asperges & vinaigrette à l'estragon
Saumon & asperges rôtis,27 minutes,"[['1', 'saumon (frais)'], ['250 g', 'asperges vertes (fraîches)'], ['1/8', 'citron jaune']]",saumon & asperges rôtis
Tarte fine aux asperges,plat froid,"[['1 bou.', 'asperges vertes (fraîches)'], ['1', 'pâte feuilletée'], ['200 g', 'chèvre frais'], ['5 cl', 'crème liquide']]",tarte fine aux asperges
Pâtes au citron & asperges,12 minutes,"[['100 g', 'pâtes (orecchiette)'], ['1/4', 'citron jaune'], ['20 g', 'parmesan (râpé)'], ['15 g', 'beurre'], ['1/4 bou.', 'asperges vertes (fraîches)']]",pâtes au citron & asperges
"Asperges, ricotta & pistaches",plat froid,"[['80 g', 'asperges vertes (fraîches)'], ['20 g', 'pistaches (grillées)'], ['1/4', 'citron jaune'], ['3 càs', 'ricotta'], ['facultatif', '1/4 bou.', 'menthe (feuilles)'], ['1 càs', 'vinaigre balsamique']]","asperges, ricotta & pistaches"
Asperges & sauce mousseline,6 minutes,"[['200 g', 'asperge blanche (fraîche)'], ['1', 'oeuf'], ['1 càs', 'moutarde'], ['5 cl', 'huile de colza'], ['1 càs', 'vinaigre de cidre']]",asperges & sauce mousseline
Salade d’asperges & œuf mollet,22 minutes,"[['1/2 bou.', 'asperges vertes (fraîches)'], ['40 g', 'petits pois (frais)'], ['10 g', 'parmesan (morceaux)'], ['1', 'oeuf'], ['30 g', 'pain (baguette)'], ['1/2', 'ail'], ['1 càs', ""huile d'olive""], ['1/4', 'citron jaune']]",salade d’asperges & oeuf mollet


In [124]:
jow_data[jow_data["recettes"].str.contains("artichaut")]

,cuisson,ingredients,recettes
Salade artichauts & noisettes,plat froid,"[['120 g', 'artichauts marinés'], ['2 poignées', 'salade (roquette)'], ['40 g', 'parmesan (morceaux)'], ['10 g', 'noisette']]",salade artichauts & noisettes
Artichaut vinaigrette,40 minutes,"[['1', 'artichaut (gros)'], ['1/2 càs', ""huile d'olive""], ['1 càc', 'vinaigre de vin rouge'], ['1 càc', ""moutarde à l'ancienne""]]",artichaut vinaigrette
Salade de pâtes & artichauts,10 minutes,"[['100 g', 'pâtes (conchiglioni)'], ['80 g', 'artichauts marinés'], ['50 g', 'olive noire'], ['1 poignée', 'salade (roquette)']]",salade de pâtes & artichauts
"Artichauts sautés, parmesan & citron",15 minutes,"[['250 g', 'artichaut (petit)'], ['15 g', 'parmesan (morceaux)'], ['1/4', 'citron jaune'], ['facultatif', '1 pinc.', 'thym (branches)']]","artichauts sautés, parmesan & citron"
Quiche artichaut & parma,50 minutes,"[['1', 'pâte brisée'], ['1', 'mozzarella (boule)'], ['160 g', 'artichauts marinés'], ['4 tran.', 'jambon cru (prosciutto)'], ['3', 'oeuf'], ['10 cl', 'lait'], ['10 cl', 'crème liquide']]",quiche artichaut & parma
Pizza chorizo & artichaut,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['1', 'mozzarella (boule)'], ['6 tran.', 'chorizo (tranches)'], ['80 g', 'artichauts marinés'], ['facultatif', '1/5 bou.', 'basilic (frais)']]",pizza chorizo & artichaut
Artichauts braisés & aïoli au citron confit,30 minutes,"[['2', 'artichaut (petit)'], ['20 g', 'citron confit'], ['1/2 gou.', 'ail'], ['1/4', 'orange'], ['1/4', 'oeuf'], ['5 cl', 'huile de colza'], ['1/2 càc', 'origan (séché)'], ['1/2 càc', 'moutarde']]",artichauts braisés & aïoli au citron confit
One pot pasta artichauts & tomates séchées,13 minutes,"[['100 g', 'pâtes (trofie)'], ['1/4', 'échalote'], ['50 g', 'tomate séchée'], ['80 g', 'artichauts marinés'], ['20 g'], ['facultatif', '1/10 bou.']]",one pot pasta artichauts & tomates séchées


In [125]:
menu_data.loc[41, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='saumon & asperges rôtis'].index[0]

Next recipe

In [126]:
menu_data[menu_data.index==42]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
42,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Hox Signatures,Hox Cheeseburger,"Spanish beef, cheddar, caramelized onions, fries, salad",Not yet handled


In [127]:
jow_data[jow_data["recettes"].str.contains("cheeseburger")]

,cuisson,ingredients,recettes
Classic Cheeseburger,40 minutes,"[['1/2 càc', 'yellow mustard'], ['1 càc', 'ketchup'], ['1/8', 'oignon rouge'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)']]",classic cheeseburger
Cheeseburger veggie,4 minutes,"[['1', 'pain burger'], ['1', 'steak végétal'], ['1/2 poignée', 'salade (coeur de laitue)'], ['1 càc', 'yellow mustard'], ['1 càs', 'ketchup'], ['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)']]",cheeseburger veggie


In [128]:
jow_data[jow_data["recettes"].str.contains("burger")]

,cuisson,ingredients,recettes
Blue Burger,40 minutes,"[['1 càc', ""confits d'oignons""], ['30 g', 'bleu'], ['1', 'boeuf (steak haché)'], ['150 g', 'patate douce'], ['1 poignée', 'salade (roquette)'], ['1', 'pain burger'], ['1/2 càc', ""moutarde à l'ancienne""]]",blue burger
French Burger,40 minutes,"[['2 càc', ""confits d'oignons""], ['30 g', 'comté'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1/2 càc', ""moutarde à l'ancienne""]]",french burger
Italian burger,40 minutes,"[['1/2 càs', 'sauce pesto'], ['1/2', 'mozzarella (boule)'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['1', 'pain burger'], ['1/4', 'tomate'], ['facultatif', '1 pinc.', 'origan (séché)'], ['1 poignée', 'salade (roquette)']]",italian burger
Salmon burger,40 minutes,"[['1/2 càs', 'crème fraîche épaisse'], ['1/8', 'citron jaune'], ['1 poignée', 'epinard (frais)'], ['1/2', 'avocat'], ['1', 'saumon (frais)'], ['1', 'pain burger'], ['150 g', 'pomme de terre'], ['facultatif', '1/10 bou.', 'ciboulette']]",salmon burger
Classic Cheeseburger,40 minutes,"[['1/2 càc', 'yellow mustard'], ['1 càc', 'ketchup'], ['1/8', 'oignon rouge'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)']]",classic cheeseburger
Super Bacon Burger,40 minutes,"[['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)'], ['1', 'boeuf (steak haché)'], ['150 g', 'pomme de terre'], ['10 g', 'salade (coeur de laitue)'], ['1', 'pain burger'], ['1 càc', 'moutarde'], ['1 càs', ""sirop d'érable""]]",super bacon burger
Tom burger,40 minutes,"[['1 càc', ""confits d'oignons""], ['1/2 càs', 'crème fraîche épaisse'], ['1/4', 'échalote'], ['150 g', 'pomme de terre'], ['1', 'pain burger'], ['30 g', 'tomme'], ['1', 'boeuf (steak haché)'], ['1/10 bou.', 'persil (frais)']]",tom burger
Brunch Burger,3 minutes,"[['1', 'pain burger'], ['2', 'oeuf'], ['15 g', 'beurre'], ['facultatif', '1/10 bou.', 'ciboulette'], ['1 tran.', 'jambon blanc']]",brunch burger
Cheeseburger veggie,4 minutes,"[['1', 'pain burger'], ['1', 'steak végétal'], ['1/2 poignée', 'salade (coeur de laitue)'], ['1 càc', 'yellow mustard'], ['1 càs', 'ketchup'], ['1/8', 'oignon rouge'], ['1 tran.', 'cheddar (tranches)']]",cheeseburger veggie
Burger raclette,5 minutes,"[['1', 'pain burger (brasserie)'], ['1', 'boeuf (steak haché)'], ['2 càc', ""confits d'oignons""], ['1 càc', ""moutarde à l'ancienne""], ['20 g', 'raclette']]",burger raclette


In [129]:
menu_data.loc[42, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='classic cheeseburger'].index[0]

Next recipe

In [130]:
menu_data[menu_data.index==43]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
43,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Hox Signatures,Beyond meat veggie burger,"Beyond Meat pattie, charcoal bun, avocado, fries, salad",Not yet handled


In [131]:
menu_data.loc[43, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='cheeseburger veggie'].index[0]

Next recipe

In [132]:
menu_data[menu_data.index==44]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
44,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Sides,Frites,(v),Not yet handled


In [133]:
jow_data[jow_data["recettes"].str.contains("frite")]

,cuisson,ingredients,recettes


In [134]:
jow_data[jow_data["ingredients"].str.contains("frite")]

,cuisson,ingredients,recettes


In [135]:
menu_data.loc[44, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [136]:
menu_data[menu_data.index==45]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
45,feaa6c7b-9b16-4d51-b71d-75518f16cb08.jpg,Sides,Frites patate douce,"Parmesan cheese, rosemary, spicy mayo",Not yet handled


In [137]:
menu_data.loc[45, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [138]:
menu_data[menu_data.index==46]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
46,in-bocca-al-lupo_42356_293.jpg,Calzoni,Calzone parigino,"mozzarella, jambon, champignons, œuf, sauce tomates au dessus",Not yet handled


In [139]:
jow_data[jow_data["recettes"].str.contains("calzone")]

,cuisson,ingredients,recettes


In [140]:
jow_data[jow_data["recettes"].str.contains("pizza")]

,cuisson,ingredients,recettes
Pizza Parma,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['3 tran.', 'jambon cru'], ['1', 'mozzarella (boule)'], ['1 poignée', 'salade (roquette)']]",pizza parma
Pizza maison 3 fromages,10 minutes,"[['1', 'pâte à pizza'], ['50 g', 'roquefort'], ['50 g', 'tomate (concentré)'], ['1', 'mozzarella (boule)'], ['100 g', 'chèvre frais']]",pizza maison 3 fromages
Pizza maison chèvre miel,10 minutes,"[['1', 'pâte à pizza'], ['2 g', 'romarin (séché)'], ['2 càc', 'miel (liquide)'], ['100 g', 'chèvre frais']]",pizza maison chèvre miel
Pizza verde à la courgette,15 minutes,"[['1', 'pâte à pizza'], ['3 càs', 'sauce pesto'], ['1', 'courgette'], ['1', 'mozzarella (boule)'], ['2 poignées', 'epinard (frais)']]",pizza verde à la courgette
Pizza verde à la burrata,10 minutes,"[['1', 'pâte à pizza'], ['1', 'burrata'], ['100 g', 'artichauts marinés'], ['1 poignée', 'salade (roquette)'], ['facultatif', '1 càs', 'pignons de pin'], ['2 càs', 'sauce pesto'], ['1 càs', 'olives vertes']]",pizza verde à la burrata
Pizza printanière,10 minutes,"[['1', 'pâte à pizza'], ['140 g', 'petits pois (surgelés)'], ['4 tran.', 'jambon cru (prosciutto)'], ['1', 'oeuf'], ['1 càs', 'sauce pesto'], ['1', 'mozzarella (boule)'], ['facultatif', '2 poignées', 'salade (roquette)']]",pizza printanière
Pizza saumon avocat,8 minutes,"[['1', 'pâte à pizza'], ['1', 'avocat'], ['4 tran.', 'saumon (fumé)'], ['80 g', 'fromage frais'], ['2 poignées', 'salade (roquette)'], ['1/2', 'citron jaune']]",pizza saumon avocat
Pizza poire gorgonzola,10 minutes,"[['1', 'poire'], ['1', 'pâte à pizza'], ['100 g', 'gorgonzola'], ['facultatif', '20 g', 'noisette'], ['2 càs', 'crème fraîche épaisse']]",pizza poire gorgonzola
Pizza puttanesca,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['60 g', 'anchois'], ['60 g', 'olive noire'], ['1 càs', 'câpres'], ['1', 'mozzarella (boule)']]",pizza puttanesca
"Pizza pesto, tomates & burrata",10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'sauce pesto'], ['200 g', 'tomate cerise'], ['1', 'burrata'], ['facultatif', '1/5 bou.', 'basilic (frais)']]","pizza pesto, tomates & burrata"


In [141]:
menu_data.loc[46, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [142]:
menu_data[menu_data.index==49]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
49,in-bocca-al-lupo_42356_293.jpg,Pizza Bambini (pour les petits),Prosciutto,"sauce tomate, mozzarella, jambon",Not yet handled


In [143]:
menu_data.loc[49, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza parma'].index[0]

Next recipe

In [144]:
menu_data[menu_data.index==50]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
50,in-bocca-al-lupo_42356_293.jpg,Planches,Salumi misti,assortiment de charcuteries italiennes,Not yet handled


In [145]:
jow_data[jow_data["recettes"].str.contains("charcuterie")]

,cuisson,ingredients,recettes
Planche de charcuterie,plat froid,"[['1 tran.', 'jambon blanc'], ['1 tran.', 'jambon cru'], ['2 tran.', 'chorizo (tranches)'], ['2 tran.', 'rosette'], ['20 g', 'rillettes'], ['10 g', 'cornichon']]",planche de charcuterie


In [146]:
menu_data.loc[50, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='planche de charcuterie'].index[0]

Next recipe

In [147]:
menu_data[menu_data.index==51]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
51,in-bocca-al-lupo_42356_293.jpg,Insalata,Mista,"roquette, mozzarella di bufala, artichauts, jambon de parme, tomates cerises, parmesan",Not yet handled


In [148]:
jow_data[jow_data["recettes"].str.contains("jambon")]

,cuisson,ingredients,recettes
Les classiques coquillettes au jambon,7 minutes,"[['100 g', 'pâtes (coquillettes)'], ['1 tran.', 'jambon blanc']]",les classiques coquillettes au jambon
Tartine chèvre figue jambon,10 minutes,"[['2', 'figue'], ['2 tran.', 'pain de campagne (tranché)'], ['2 càs', 'chèvre frais'], ['2 tran.', 'jambon cru'], ['1 càc', 'miel (liquide)'], ['10 g', 'noix'], ['1 poignée', 'salade (mélange)']]",tartine chèvre figue jambon
Gratin de pâtes au jambon,25 minutes,"[['100 g', 'pâtes (macaroni)'], ['40 g', 'fromage râpé'], ['1 tran.', 'jambon blanc'], ['1 càs', 'crème fraîche épaisse']]",gratin de pâtes au jambon
Poireaux gratinés au jambon,plat froid,"[['1', 'poireau'], ['1 tran.', 'jambon blanc'], ['10 cl', 'crème liquide'], ['25 g', 'fromage râpé']]",poireaux gratinés au jambon
Brochette melon & jambon,plat froid,"[['facultatif', '4', 'pics à brochette'], ['1/4', 'melon'], ['2 tran.', 'jambon cru (prosciutto)'], ['1/10 bou.', 'basilic (frais)'], ['60 g', 'mozzarella (mini)']]",brochette melon & jambon
Melon jambon 2.0,plat froid,"[['1/2', 'melon'], ['80 g', 'ricotta'], ['3 tran.', 'jambon cru'], ['1/10 bou.', 'basilic (frais)']]",melon jambon 2.0
Soupe potimarron & jambon,plat froid,"[['1/2', 'potimarron'], ['5 cl', 'crème liquide'], ['1 tran.', 'jambon cru'], ['1/2', 'échalote']]",soupe potimarron & jambon
Endives caramélisées au jambon,20 minutes,"[['250 g', 'endives'], ['2 tran.', 'jambon cru'], ['1 càs', 'crème fraîche épaisse'], ['1 càs', 'parmesan (râpé)']]",endives caramélisées au jambon
Pâtes au beaufort & jambon,10 minutes,"[['130 g', 'pâtes (fusilli)'], ['1 tran.', 'jambon blanc'], ['5 cl', 'crème liquide'], ['40 g', 'beaufort']]",pâtes au beaufort & jambon
Purée maison & jambon,plat froid,"[['1 tran.', 'jambon blanc'], ['250 g', 'pomme de terre'], ['5 cl', 'lait'], ['20 g', 'beurre'], ['1 pinc.']]",purée maison & jambon


In [149]:
jow_data[jow_data["recettes"].str.contains("parma")]

,cuisson,ingredients,recettes
Pizza Parma,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['3 tran.', 'jambon cru'], ['1', 'mozzarella (boule)'], ['1 poignée', 'salade (roquette)']]",pizza parma
Insalata di parma,plat froid,"[['2 poignées', 'salade (mélange)'], ['1/4', 'melon'], ['100 g', 'concombre'], ['1 tran.', 'jambon cru (prosciutto)'], ['facultatif', '1/10 bou.', 'ciboulette']]",insalata di parma
Galette parma,3 minutes,"[['15 g', 'tomate séchée'], ['1', 'galette bretonne'], ['1/2', 'mozzarella (boule)'], ['1/4 càs', 'sauce pesto'], ['1 tran.', 'jambon cru']]",galette parma
Bruschetta parma,plat froid,"[['2 tran.', 'pain de campagne (tranché)'], ['50 g', 'mozzarella (boule)'], ['150 g', 'tomate'], ['facultatif', '1/10 bou.', 'basilic (frais)'], ['2 tran.', 'jambon cru']]",bruschetta parma
Quiche artichaut & parma,50 minutes,"[['1', 'pâte brisée'], ['1', 'mozzarella (boule)'], ['160 g', 'artichauts marinés'], ['4 tran.', 'jambon cru (prosciutto)'], ['3', 'oeuf'], ['10 cl', 'lait'], ['10 cl', 'crème liquide']]",quiche artichaut & parma
"Salade nectarine, parma & mozzarella",plat froid,"[['1', 'nectarine'], ['1/2', 'avocat'], ['1 tran.', 'jambon cru'], ['2 poignées', 'salade (roquette)'], ['1 càs'], ['1/2']]","salade nectarine, parma & mozzarella"
Poulet parma & salade,15 minutes,"[['1', 'poulet (pané)'], ['50 g', 'sauce tomate (basilic)'], ['1 càs', 'parmesan (râpé)'], ['40 g', 'mozzarella (à cuire)'], ['2 poignées', 'salade (mélange)'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",poulet parma & salade
"Salade lentilles, brocolis & parma",10 minutes,"[['130 g', 'lentilles (cuites)'], ['100 g', 'brocoli (frais)'], ['2 tran.', 'jambon cru'], ['1/4', 'échalote'], ['1/2 càc', ""moutarde à l'ancienne""]]","salade lentilles, brocolis & parma"
Aubergine parma & salade,35 minutes,"[['1/2', 'aubergine'], ['50 g', 'sauce tomate (basilic)'], ['40 g', 'mozzarella (boule)'], ['2 poignées', 'salade (mélange)'], ['1 càs', 'parmesan (râpé)'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",aubergine parma & salade


There is only one salad with ham

In [150]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='salade jambon cru, mozza & pistaches'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='insalata di parma'].index[0])

menu_data.loc[51, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [151]:
menu_data[menu_data.index==52]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
52,in-bocca-al-lupo_42356_293.jpg,Desserts,Tiramisù,NaN,Not yet handled


In [152]:
jow_data[jow_data["recettes"].str.contains("tiramis")]

,cuisson,ingredients,recettes
Tiramisu,3 heures,"[['100 g', 'sucre (en poudre)'], ['facultatif', '1 sac.', 'sucre vanillé'], ['3', 'oeuf'], ['2 càs', 'cacao (en poudre)'], ['250 g', 'mascarpone'], ['20', 'biscuit cuiller'], ['2 càc', 'café'], ['facultatif', '1 càs', 'amaretto']]",tiramisu
Roulé façon tiramisu,12 minutes,"[['4', 'oeuf'], ['125 g', 'farine de blé'], ['125 g', 'sucre (en poudre)'], ['1 càc', 'levure chimique'], ['35 g'], ['25 cl'], ['120 g'], ['15 g']]",roulé façon tiramisu


In [153]:
menu_data.loc[52, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='tiramisu'].index[0]

Next recipe

In [154]:
menu_data[menu_data.index==53]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
53,in-bocca-al-lupo_42356_293.jpg,Desserts,Panna cotta (avec gousses de vanille) et coulis de fruits de frais,NaN,Not yet handled


In [155]:
jow_data[jow_data["recettes"].str.contains("panna cotta")]

,cuisson,ingredients,recettes
Panna cotta à la fraise,3 minutes,"[['80 g', 'fraise'], ['10 cl', 'crème liquide'], ['2 càs', 'sucre (en poudre)'], ['1/4', 'vanille (gousse)'], ['1/2', 'gélatine']]",panna cotta à la fraise


In [156]:
menu_data.loc[53, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='panna cotta à la fraise'].index[0]

Next recipe

In [157]:
menu_data[menu_data.index==55]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
55,menu_bobby.jpg,Petites assiettes à partager,Gros raviolo,aux champis d'une ferme urbaine du 18è siècle,Not yet handled


In [158]:
jow_data[jow_data["recettes"].str.contains("raviol")]

,cuisson,ingredients,recettes
Ravioles sautées,4 minutes,"[['120 g', 'ravioles au fromage'], ['2 poignées', 'epinard (frais)'], ['10 g', 'noisette']]",ravioles sautées
Bouillon de ravioles du Dauphiné,plat froid,"[['120 g', 'ravioles au fromage'], ['150 g', 'champignons (mélange)'], ['1/2', 'bouillon de volaille (cube)'], ['1/2', 'échalote']]",bouillon de ravioles du dauphiné
Raviolis à la crème de champignons,14 minutes,"[['125 g', 'ravioli (aux champignons)'], ['1 càs', 'crème fraîche épaisse'], ['100 g', 'champignons de paris (frais)'], ['1/4 gou.', 'ail'], ['facultatif', '1/2 càs', 'cognac'], ['1/2 càs', 'parmesan (râpé)']]",raviolis à la crème de champignons
Canard & ravioles forestières,15 minutes,"[['1/2', 'canard (magret)'], ['5 ml', 'vin blanc'], ['10 g', 'parmesan (morceaux)'], ['125 g', 'ravioli (aux champignons)']]",canard & ravioles forestières
"Ravioli, tomates séchées & câpres",10 minutes,"[['125 g', 'ravioli (ricotta-épinards)'], ['40 g', 'tomate séchée'], ['1 poignée', 'epinard (frais)'], ['1/2 càs', 'câpres'], ['50 g', 'artichauts marinés'], ['10 g', 'parmesan (morceaux)']]","ravioli, tomates séchées & câpres"
Ravioles fromage & poire,5 minutes,"[['125 g', 'ravioli (au gorgonzola)'], ['10 cl', 'crème liquide'], ['5 g', 'noix'], ['1/2 càs', 'parmesan (râpé)'], ['1/3', 'poire']]",ravioles fromage & poire
Gratin de ravioles au saumon,30 minutes,"[['120 g', 'ravioles au fromage'], ['1', 'poireau'], ['1 tran.', 'saumon (fumé)'], ['10 cl', 'crème liquide'], ['20 g', 'comté']]",gratin de ravioles au saumon
Salade de ravioli ricotta et épinards,2 minutes,"[['125 g', 'ravioli (ricotta-épinards)'], ['2 poignées', 'salade (mélange)'], ['15 g', 'comté'], ['10 g', 'pignons de pin'], ['1 càc', 'vinaigre balsamique']]",salade de ravioli ricotta et épinards
Ravioles à la toscane,7 minutes,"[['125 g', 'ravioli (ricotta-épinards)'], ['50 g', 'tomate séchée'], ['1 càs', 'crème fraîche épaisse'], ['2 poignées', 'epinard (frais)'], ['1 càs', 'parmesan (râpé)'], ['1/4 gou.', 'ail']]",ravioles à la toscane
Gratin de ravioles à la courgette,25 minutes,"[['150 g', 'ravioles au fromage'], ['90 g', 'crème fraîche épaisse'], ['25 g', 'comté (râpé)'], ['100 g', 'courgette']]",gratin de ravioles à la courgette


In [159]:
tmp = []
tmp.append(jow_data[jow_data["recettes"]=='raviolis à la crème de champignons'].index[0])
tmp.append(jow_data[jow_data["recettes"]=='raviolis aux champignons, chèvre frais & noisette'].index[0])

menu_data.loc[55, 'Closest JOW recipe by hand'] = tmp

Next recipe

In [160]:
menu_data[menu_data.index==57]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
57,menu_bobby.jpg,Pizz',Quattro,"Mozza fior di latte et scamorza IDF, gorgonzola, parmigiano, basilico",Not yet handled


In [161]:
jow_data[jow_data["recettes"].str.contains("pizza")]

,cuisson,ingredients,recettes
Pizza Parma,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['3 tran.', 'jambon cru'], ['1', 'mozzarella (boule)'], ['1 poignée', 'salade (roquette)']]",pizza parma
Pizza maison 3 fromages,10 minutes,"[['1', 'pâte à pizza'], ['50 g', 'roquefort'], ['50 g', 'tomate (concentré)'], ['1', 'mozzarella (boule)'], ['100 g', 'chèvre frais']]",pizza maison 3 fromages
Pizza maison chèvre miel,10 minutes,"[['1', 'pâte à pizza'], ['2 g', 'romarin (séché)'], ['2 càc', 'miel (liquide)'], ['100 g', 'chèvre frais']]",pizza maison chèvre miel
Pizza verde à la courgette,15 minutes,"[['1', 'pâte à pizza'], ['3 càs', 'sauce pesto'], ['1', 'courgette'], ['1', 'mozzarella (boule)'], ['2 poignées', 'epinard (frais)']]",pizza verde à la courgette
Pizza verde à la burrata,10 minutes,"[['1', 'pâte à pizza'], ['1', 'burrata'], ['100 g', 'artichauts marinés'], ['1 poignée', 'salade (roquette)'], ['facultatif', '1 càs', 'pignons de pin'], ['2 càs', 'sauce pesto'], ['1 càs', 'olives vertes']]",pizza verde à la burrata
Pizza printanière,10 minutes,"[['1', 'pâte à pizza'], ['140 g', 'petits pois (surgelés)'], ['4 tran.', 'jambon cru (prosciutto)'], ['1', 'oeuf'], ['1 càs', 'sauce pesto'], ['1', 'mozzarella (boule)'], ['facultatif', '2 poignées', 'salade (roquette)']]",pizza printanière
Pizza saumon avocat,8 minutes,"[['1', 'pâte à pizza'], ['1', 'avocat'], ['4 tran.', 'saumon (fumé)'], ['80 g', 'fromage frais'], ['2 poignées', 'salade (roquette)'], ['1/2', 'citron jaune']]",pizza saumon avocat
Pizza poire gorgonzola,10 minutes,"[['1', 'poire'], ['1', 'pâte à pizza'], ['100 g', 'gorgonzola'], ['facultatif', '20 g', 'noisette'], ['2 càs', 'crème fraîche épaisse']]",pizza poire gorgonzola
Pizza puttanesca,10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'tomate (purée)'], ['60 g', 'anchois'], ['60 g', 'olive noire'], ['1 càs', 'câpres'], ['1', 'mozzarella (boule)']]",pizza puttanesca
"Pizza pesto, tomates & burrata",10 minutes,"[['1', 'pâte à pizza'], ['60 g', 'sauce pesto'], ['200 g', 'tomate cerise'], ['1', 'burrata'], ['facultatif', '1/5 bou.', 'basilic (frais)']]","pizza pesto, tomates & burrata"


In [162]:
menu_data.loc[57, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza maison 3 fromages'].index[0]

Next recipe

In [163]:
menu_data[menu_data.index==58]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
58,menu_bobby.jpg,Pizz',Zucchine,"Crème de courgette, mozza fior di latte IDF, provolone del monaco, chips de courgette, basilico",Not yet handled


In [164]:
menu_data.loc[58, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pizza verde à la courgette'].index[0]

Next recipe

In [165]:
menu_data[menu_data.index==59]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
59,menu_bobby.jpg,Pasta Fresca,Limone,"Trenette, sauce au citron, piment (ou non si vous aimez pas ), basilico",Not yet handled


In [166]:
jow_data[jow_data["recettes"].str.contains("pâte")]

,cuisson,ingredients,recettes
Pâtes aux aubergines & parmesan,10 minutes,"[['100 g', 'pâtes (linguine)'], ['1/4', 'aubergine'], ['2 tran.', 'jambon cru'], ['20 g', 'parmesan (morceaux)']]",pâtes aux aubergines & parmesan
Gratin de pâtes au jambon,25 minutes,"[['100 g', 'pâtes (macaroni)'], ['40 g', 'fromage râpé'], ['1 tran.', 'jambon blanc'], ['1 càs', 'crème fraîche épaisse']]",gratin de pâtes au jambon
Pâtes à la Sicilienne,9 minutes,"[['100 g', 'pâtes (conchiglioni)'], ['1 càs', 'câpres'], ['100 g', 'tomate cerise'], ['90 g', 'thon (conserve)'], ['1 càc']]",pâtes à la sicilienne
Salade de pâtes printanière,10 minutes,"[['2 tran.', 'jambon cru'], ['80 g', 'pâtes (pipe rigate)'], ['1/2', 'salade (sucrine)'], ['70 g', 'petits pois (frais)'], ['1/4', 'citron jaune'], ['15 g', 'parmesan (morceaux)']]",salade de pâtes printanière
Pâtes au pesto de coriandre,8 minutes,"[['120 g', 'pâtes (pipe rigate)'], ['1/5 bou.', 'coriandre (frais)'], ['15 g', 'amandes (entières)'], ['1/4 gou.', 'ail'], ['20 g', 'parmesan (râpé)']]",pâtes au pesto de coriandre
Pâte à tartiner aux speculoos,plat froid,"[['200 g', 'speculoos'], ['1 càc', 'cannelle (en poudre)'], ['1 càc', 'miel (liquide)'], ['300 g', 'lait concentré sucré']]",pâte à tartiner aux speculoos
Pâte à tartiner au chocolat,plat froid,"[['50 g', 'noisette'], ['50 g', 'chocolat à cuire'], ['7 cl', 'crème liquide'], ['8 g', 'sucre (en poudre)'], ['5 ml', 'huile de tournesol']]",pâte à tartiner au chocolat
Pâtes au brocoli,20 minutes,"[['100 g', 'brocoli (frais)'], ['100 g', 'pâtes (orecchiette)'], ['2', 'anchois'], ['1/4 gou.', 'ail'], ['10 g', 'parmesan (morceaux)']]",pâtes au brocoli
Pâtes au beaufort & jambon,10 minutes,"[['130 g', 'pâtes (fusilli)'], ['1 tran.', 'jambon blanc'], ['5 cl', 'crème liquide'], ['40 g', 'beaufort']]",pâtes au beaufort & jambon
"Pâtes thon, citron & ricotta",12 minutes,"[['100 g', 'pâtes (linguine)'], ['1 càs', 'ricotta'], ['50 g', 'thon au citron'], ['1/4', 'citron jaune'], ['1 càs', 'crème liquide']]","pâtes thon, citron & ricotta"


In [167]:
jow_data[jow_data["recettes"].str.contains("spaghetti")]

,cuisson,ingredients,recettes
Spaghetti ail & crevette,10 minutes,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'crevette (cuite)'], ['1/2 gou.', 'ail'], ['1 pinc.', ""piment d'espelette""], ['1/10 bou.', 'persil (frais)']]",spaghetti ail & crevette
Spaghetti alla puttanesca,plat froid,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'tomate (chair)'], ['20 g', 'olive noire'], ['1/2 càs', 'câpres']]",spaghetti alla puttanesca
Spaghetti endives & prosciutto,10 minutes,"[['100 g', 'pâtes (spaghetti)'], ['100 g', 'endives'], ['1 tran.', 'jambon cru (prosciutto)'], ['1/2 càs', 'parmesan (râpé)'], ['1 càc', 'sucre (en poudre)']]",spaghetti endives & prosciutto
Spaghetti alla bolognese express,plat froid,"[['1/4', 'oignon jaune'], ['1/2', 'carotte (frais)'], ['100 g', 'tomate (purée)'], ['1/4 gou.', 'ail']]",spaghetti alla bolognese express
Cherry tomatoes spaghetti,10 minutes,"[['120 g', 'tomate cerise'], ['100 g', 'pâtes (spaghetti)'], ['15 g', 'parmesan (morceaux)'], ['1/10 bou.', 'basilic (frais)'], ['1/4 gou.', 'ail']]",cherry tomatoes spaghetti
Spaghettis nordiques,13 minutes,"[['150 g', 'pâtes (spaghetti)'], ['1 càs', 'mascarpone'], ['1/4', 'citron jaune'], ['facultatif', '1/10 bou.', 'aneth (frais)'], ['10 g', 'oeufs de poisson']]",spaghettis nordiques
Spaghetti bolognese à la ratatouille,10 minutes,"[['150 g', 'ratatouille'], ['100 g', 'boeuf (haché)'], ['1 pinc.', 'origan (séché)'], ['100 g', 'pâtes (spaghetti)'], ['1/2 càs']]",spaghetti bolognese à la ratatouille


In [168]:
jow_data[jow_data["recettes"].str.contains("tagliatelle")]

,cuisson,ingredients,recettes
Tagliatelles crevettes & chorizo,10 minutes,"[['70 g', 'pâtes (tagliatelles)'], ['80 g', 'crevette (cuite)'], ['1 càs', 'crème fraîche épaisse'], ['25 g', 'chorizo (entier)'], ['60 g', 'petits pois (surgelés)']]",tagliatelles crevettes & chorizo
Tagliatelles poulet & champignons,10 minutes,"[['1', 'poulet (escalope)'], ['100 g', 'champignons de paris (frais)'], ['70 g', 'pâtes (tagliatelles)'], ['1 càs', 'crème fraîche épaisse'], ['1/4', 'oignon jaune']]",tagliatelles poulet & champignons
Tagliatelles crémeuses & saumon,13 minutes,"[['100 g', 'pâtes fraîches'], ['1', 'saumon (frais)'], ['50 g', 'fromage ail & fines herbes'], ['1/4', 'citron jaune'], ['facultatif', '1 càs']]",tagliatelles crémeuses & saumon
Tagliatelles saumon épinards,10 minutes,"[['100 g', 'pâtes (tagliatelles)'], ['1 tran.', 'saumon (fumé)'], ['1 poignée', 'epinard (frais)'], ['1 càs', 'crème fraîche épaisse'], ['1/4', 'citron jaune']]",tagliatelles saumon épinards


In [169]:
jow_data[jow_data["recettes"].str.contains("pasta")]

,cuisson,ingredients,recettes
Pasta alla carbonara,9 minutes,"[['100 g', 'pâtes (spaghetti)'], ['3 tran.', 'pancetta'], ['1', 'oeuf'], ['40 g', 'parmesan (morceaux)']]",pasta alla carbonara
Pasta Alfredo,12 minutes,"[['100 g', 'pâtes (tagliatelles)'], ['1', 'poulet (escalope)'], ['10 cl', 'crème liquide'], ['1/2 gou.', 'ail'], ['1 càs', 'parmesan (râpé)']]",pasta alfredo
Creamy avocado pasta,8 minutes,"[['120 g', 'pâtes (linguine)'], ['1/4', 'avocat'], ['1/5 bou.', 'basilic (frais)'], ['10 g', 'amandes (effilées)'], ['15 g', 'parmesan (morceaux)']]",creamy avocado pasta
Pasta petits pois lardons,10 minutes,"[['100 g', 'pâtes (pipe rigate)'], ['100 g', 'petits pois (frais)'], ['75 g', 'lardons'], ['2 cl', 'crème liquide'], ['2 càs'], ['facultatif', '1/50 bou.']]",pasta petits pois lardons
Pasta Rocchini,15 minutes,"[['100 g', 'pâtes (trofie)'], ['2', 'saucisse (chipolata)'], ['1 poignée', 'salade (roquette)'], ['1 càs', 'parmesan (râpé)'], ['1/2', 'oignon rouge'], ['1/2', 'fenouil']]",pasta rocchini
Pasta poulet citron,10 minutes,"[['1', 'poulet (escalope)'], ['100 g', 'pâtes (spaghetti)'], ['8 cl', 'crème liquide'], ['1 poignée', 'epinard (frais)'], ['1 càs', 'parmesan (râpé)'], ['1/2', 'citron jaune']]",pasta poulet citron
Creamy Eggs & Prosciutto pasta,8 minutes,"[['100 g', 'pâtes (pappardelle)'], ['2 tran.', 'jambon cru (prosciutto)'], ['5 cl', 'crème liquide'], ['1/2 càs', 'ricotta'], ['1', 'oeuf'], ['facultatif', '1/10 bou.', 'basilic (frais)']]",creamy eggs & prosciutto pasta
Greek pasta salad,8 minutes,"[['70 g', 'pâtes (fusilli)'], ['1', 'tomate'], ['15 g', 'olive noire'], ['130 g', 'concombre'], ['30 g', 'feta'], ['facultatif', '1/8', 'oignon rouge'], ['1 pinc.', 'origan (séché)']]",greek pasta salad
Pasta pistacchio e olive,plat froid,"[['15 g', 'pistaches (grillées)'], ['100 g', 'pâtes (pipe rigate)'], ['15 g', 'olive noire'], ['1/4 gou.', 'ail']]",pasta pistacchio e olive
Pasta al ragu d'anatra,plat froid,"[['100 g', 'pâtes (pappardelle)'], ['1/2', 'canard (confit)'], ['70 g', 'tomate pelée'], ['1/2', 'échalote']]",pasta al ragu d'anatra


In [170]:
menu_data.loc[59, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pasta al limone'].index[0]

Next recipe

In [171]:
menu_data[menu_data.index==60]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
60,menu_bobby.jpg,Pasta Fresca,Norma,"Tagliatelle, sauce tomate et aubergines frites, ricotta salée fumée",Not yet handled


In [172]:
menu_data.loc[60, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pasta alla norma'].index[0]

Next recipe

In [173]:
menu_data[menu_data.index==61]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
61,menu_bobby.jpg,Pasta Fresca,Carbobby,"Spaghetti, crème d'œuf bio et de pecorino, guanciale, poivre",Not yet handled


In [174]:
jow_data[jow_data["recettes"].str.contains("carbonara")]

,cuisson,ingredients,recettes
Pasta alla carbonara,9 minutes,"[['100 g', 'pâtes (spaghetti)'], ['3 tran.', 'pancetta'], ['1', 'oeuf'], ['40 g', 'parmesan (morceaux)']]",pasta alla carbonara


In [175]:
menu_data.loc[61, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='pasta alla carbonara'].index[0]

Next recipe

In [176]:
menu_data[menu_data.index==62]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
62,menu_bobby.jpg,Dolci,Affogato al caffè,Boule de glace à la vanille + shot d'espresso,Not yet handled


In [177]:
jow_data[jow_data["recettes"].str.contains("glace")]

,cuisson,ingredients,recettes
"Ananas rôti, glace coco",9 minutes,"[['1/4', 'ananas (entier)'], ['1/4', 'citron vert'], ['20 g', 'sucre (en poudre)'], ['20 g', 'beurre'], ['60 g', 'glace coco']]","ananas rôti, glace coco"
Banane caramélisée miel & glace vanille,7 minutes,"[['1', 'banane'], ['1 càs', 'miel (liquide)'], ['15 g', 'noix'], ['40 g', 'crème glacée vanille'], ['10 g']]",banane caramélisée miel & glace vanille
Gaufres au sucre glace,3 minutes,"[['50 g', 'farine de blé'], ['20 g', 'sucre (en poudre)'], ['20 g', 'beurre'], ['6 cl', 'lait'], ['1', 'oeuf'], ['2 g', 'levure chimique'], ['1 càs', 'sucre glace']]",gaufres au sucre glace
"Glace au yaourt, pêche & granola",3 heures,"[['1', 'yaourt nature'], ['1', 'pêche'], ['30 g', 'granola']]","glace au yaourt, pêche & granola"
Nectarine rôtie & glace vanille,4 minutes,"[['1 boule', 'crème glacée vanille'], ['1', 'nectarine'], ['15 g', 'amandes (effilées)'], ['20 g', 'beurre'], ['1 càc', 'sucre de canne']]",nectarine rôtie & glace vanille


In [178]:
menu_data.loc[62, 'Closest JOW recipe by hand'] = 'no match'

Next recipe

In [179]:
menu_data[menu_data.index==63]

,Menu filename,Menu Title,Menu Subtitle,Menu Ingredients,Closest JOW recipe by hand
63,menu_bobby.jpg,Dolci,Boozy tiramisù,Tiramisù classique + Amaretto (liqueur d'amande),Not yet handled


In [180]:
menu_data.loc[63, 'Closest JOW recipe by hand'] = jow_data[jow_data["recettes"]=='tiramisu'].index[0]

### To be continued with more recipes. Now let's save the file

In [181]:
menu_data[menu_data['Closest JOW recipe by hand']!='Not yet handled'].to_excel("../data/Tools/MenuToJowRecipes_v1.xlsx")

In [188]:
print("Number of menu recipes with no found match in Jow = ")
menu_data[menu_data['Closest JOW recipe by hand']=='no match']['Closest JOW recipe by hand'].count()

Number of menu recipes with no found match in Jow = 


18

In [189]:
print("Number of menu recipes that have not yet been handled = ")
menu_data[menu_data['Closest JOW recipe by hand']=='Not yet handled']['Closest JOW recipe by hand'].count()

Number of menu recipes that have not yet been handled = 


33

In [192]:
print("Number of menu recipes with found matches in Jow = ")
menu_data[(menu_data['Closest JOW recipe by hand']!='Not yet handled') & (menu_data['Closest JOW recipe by hand']!='no match')]['Closest JOW recipe by hand'].count()

Number of menu recipes with found matches in Jow = 


30